# Twitter

In this notebook we are going to show all the used for the analysis. Also we are going to show all the graphics asociated with the data obtaines.


## Introduction

We downloaded data from twitter with various methods, which did not provide the same amount of data, so we standarized to the minimal amount which it could be useful. Those fields are: 
* Date when it was created at.
* ID of the tweet, which is unique.
* The text of the tweet.
* The user, which contains the user screen name (username) and the user ID.

After defining that the period of downloading data was finished, the amount of data gathered was: **6.542.718** unique tweets. Which is a decent amount of data to analyse.

## Analysis

1) The first step is to filter the data in different files that will be used for all the analysis. The filter will be keywords (in the meantime), the keywords are the platforms name (i.e. Nintendo, Playstation, Xbox), these keywords are searched in the text and the username. If there is no defined preference (not zero preference) in which platform the record was pointing to, it is inserted in both files.

The ideal method to filter the data would be create a database of keywords asociated with each platform, so in every record when searching it could be calculated the probability of that text (according to all the words) to which platform it goes. This would require a model with N-Bayes, but like most predictive models it requires training, which we do not have at this moment.

**Note**: Also from here on we called each file as follows:
* project_tweets01.data -> Nintendo
* project_tweets02.data -> Playstation
* project_tweets03.data -> Xbox
* project_tweets04.data -> Else (Which is everything else that did not fit in the other categories)

In [ ]:
import time
import json

if __name__ == '__main__':
    time_start = time.clock()
    nintendo_platform_file = []
    playstation_platform_file = []
    xbox_platform_file = []
    else_platform_file = []
    _file = open("project_tweets.data", 'r')
    for line in _file.readlines():
        data = json.loads(line)
        try:
            raw_text = data['text'].strip().upper()
            raw_username = data['user']['screen_name'].strip().upper()
        except:
            # No idea why would happen.
            print "WTF!"
            raw_text = ""
            raw_username = ""
        nintendo_mentions = raw_text.count("NINTENDO")
        playstation_mentions = raw_text.count("PLAYSTATION")
        xbox_mentions = raw_text.count("XBOX")
        nintendo_mentions += 1 if raw_username.find("NINTENDO") != -1 else 0
        playstation_mentions += 1 if raw_username.find("PLAYSTATION") != -1 else 0
        xbox_mentions += 1 if raw_username.find("XBOX") != -1 else 0
        if nintendo_mentions > playstation_mentions and nintendo_mentions > xbox_mentions:
            nintendo_platform_file.append(line)
        elif playstation_mentions > nintendo_mentions and playstation_mentions > xbox_mentions:
            playstation_platform_file.append(line)
        elif xbox_mentions > playstation_mentions and xbox_mentions > nintendo_mentions:
            xbox_platform_file.append(line)
        elif nintendo_mentions == playstation_mentions and nintendo_mentions == xbox_mentions and nintendo_mentions > 0:
            nintendo_platform_file.append(line)
            playstation_platform_file.append(line)
            xbox_platform_file.append(line)
        elif nintendo_mentions == playstation_mentions and nintendo_mentions != xbox_mentions:
            nintendo_platform_file.append(line)
            playstation_platform_file.append(line)
        elif nintendo_mentions == xbox_mentions and nintendo_mentions != playstation_mentions:
            nintendo_platform_file.append(line)
            xbox_platform_file.append(line)
        elif playstation_mentions == xbox_mentions and playstation_mentions != nintendo_mentions:
            playstation_platform_file.append(line)
            xbox_platform_file.append(line)
        else:
            else_platform_file.append(line)
    filenames = ["project_tweets01.data", "project_tweets02.data", "project_tweets03.data", "project_tweets04.data"]
    for filename in filenames:
        # Clean files.
        open(filename, 'w').close()
        # Write files in append mode.
        with open(filename, 'a') as _file:
            if filename == "project_tweets01.data":
                for line in nintendo_platform_file:
                    _file.write(line)
            elif filename == "project_tweets02.data":
                for line in playstation_platform_file:
                    _file.write(line)
            elif filename == "project_tweets03.data":
                for line in xbox_platform_file:
                    _file.write(line)
            else:
                for line in else_platform_file:
                    _file.write(line)
    time_end = time.clock()

    print "Time taken to completion of the metric: {0} in processor time".format(time_end - time_start)


2) The first analysis to do with this data that is now filtered by platforms, is to count the amount of records per platform. This is done with the next script.

In [ ]:
import time
import sys

if __name__ == '__main__':
    time_start = time.clock()
    # Clean File.
    open("records_by_platform_summary.txt", 'w').close()
    files = ["project_tweets01.data", "project_tweets02.data", "project_tweets03.data", "project_tweets04.data", "project_tweets.data"]
    for _file in files:
        # Write to file in append mode.
        _fo = open("records_by_platform_summary.txt", 'a')
        sys.stdout = _fo
        print _file
        _data = open(_file, 'r')
        lines = _data.readlines()
        records = len(lines)
        print records, '\n'

    time_end = time.clock()

    print "Time taken to completion of the metric: {0} in processor time".format(time_end - time_start)


**Results**

project_tweets01.data

1317667 

project_tweets02.data

2337881 

project_tweets03.data

2260157 

project_tweets04.data

735410 

project_tweets.data

6542718 

Time taken to completion of the metric: 37.615491 in processor time

**Analysis**

From this we can see that in the "else" category we have **11.24%** of all the data, which is not a small amount. But considering that our filter for the platforms is kind of brute force is all right.

3) The next analysis to do is to count the amount of unique users per platform. This is done with the next script.

In [ ]:
import time
import sys

if __name__ == '__main__':
    time_start = time.clock()
    # Clean File.
    open("user_amount_by_platform_summary.txt", 'w').close()
    files = ["project_tweets01.data", "project_tweets02.data", "project_tweets03.data", "project_tweets04.data"]
    for _file in files:
        # parameters for mrjob.
        # To run your job in multiple subprocesses with a few Hadoop features simulated, use -r local.
        option1 = "" #""-r"
        option2 = "" #""local"
        sys.argv = ['user_amount.py', option1, option2, _file]
        # Write to file in append mode.
        _fo = open("user_amount_by_platform_summary.txt", 'a')
        sys.stdout = _fo
        print _file
        execfile('user_amount.py')
        print "\n"

    time_end = time.clock()

    print "Time taken to completion of the metric: {0} in processor time".format(time_end - time_start)


In [ ]:
from mrjob.job import MRJob
from mrjob.step import MRStep
from mrjob.protocol import JSONValueProtocol
import time
import itertools
import sys

class MRWordFrequencyCount(MRJob):
    INPUT_PROTOCOL = JSONValueProtocol

    def mapper(self, _, record):
        yield [record['user']['screen_name'], 1]

    def reducer(self, key, values):
        yield [key, 1]

    def mapper2(self, key, values):
        yield ['amount_users', values]

    def reducer2(self, key, values):
        yield [key, sum(values)]

    def steps(self):
        return [MRStep(mapper=self.mapper, reducer=self.reducer),
                MRStep(mapper=self.mapper2, reducer=self.reducer2)]


if __name__ == '__main__':
    #time_start = time.clock()
    MRWordFrequencyCount().run()
    #time_end = time.clock()
    #print "Time taken to completion of the metric: {0} in processor time".format(time_end - time_start)


**Results**

project_tweets01.data

"amount_users"	426450


project_tweets02.data

"amount_users"	574308


project_tweets03.data

"amount_users"	727273


project_tweets04.data

"amount_users"	265827


Time taken to completion of the metric: 426.030981 in processor time


**Analysis**

From this we can see that in the Xbox platform there are more unique users than in all the other platforms by a not small percentage, it almost duplicates Nintendo unique users. What is interesting, is using the analysis from before we can see that Xbox had less records than Playstation, but seeing this there is a whooping 150k (estimated) more unique users in Xbox, which could lead us that Playstation content creation is more for each unique user or that there is a tiny amount of users that produce all the content for this platform.

4) The next analysis is to calculate the Top 10 users that generate the most content in each platform. This is done with the next script.

In [ ]:
import time
import sys

if __name__ == '__main__':
    time_start = time.clock()
    # Clean File.
    open("top_users_by_platform_summary.txt", 'w').close()
    files = ["project_tweets01.data", "project_tweets02.data", "project_tweets03.data", "project_tweets04.data"]
    for _file in files:
        # parameters for mrjob.
        # To run your job in multiple subprocesses with a few Hadoop features simulated, use -r local.
        option1 = "" #""-r"
        option2 = "" #""local"
        sys.argv = ['top_users.py', option1, option2, _file]
        # Write to file in append mode.
        _fo = open("top_users_by_platform_summary.txt", 'a')
        sys.stdout = _fo
        print _file
        execfile('top_users.py')
        print "\n"

    time_end = time.clock()

    print "Time taken to completion of the metric: {0} in processor time".format(time_end - time_start)


In [ ]:
from mrjob.job import MRJob
from mrjob.step import MRStep
from mrjob.protocol import JSONValueProtocol
import time
import itertools
import operator
import sys

class MRWordFrequencyCount(MRJob):
    INPUT_PROTOCOL = JSONValueProtocol

    def mapper(self, _, record):
        yield [record['user']['screen_name'], 1]

    def reducer(self, key, values):
        yield ["top_user", (sum(values), key)]

    def reducer2(self, key, values):
        user_ids = []
        user_tweets = []
        for value in values:
            user_ids.append(value[1])
            user_tweets.append(value[0])
        user = {}
        for i in xrange(0, len(user_ids)):
            user[user_ids[i]] = user_tweets[i]
        top_users = sorted(user.items(), key=lambda x: (x[1], operator.itemgetter(0)), reverse=True)
        for user in top_users[0:10]:
            #print user[0], user[1]
            yield [user[0], user[1]]

    def steps(self):
        return [MRStep(mapper=self.mapper, reducer=self.reducer),
                MRStep(reducer=self.reducer2)]


if __name__ == '__main__':
    #time_start = time.clock()
    MRWordFrequencyCount().run()
    #time_end = time.clock()
    #print "Time taken to completion of the metric: {0} in processor time".format(time_end - time_start)


**Results**

project_tweets01.data
* "savetimeandmoey"	11391
* "AuctionPorn"	8449
* "AmazonBay4u"	8352
* "retrodeals"	7435
* "Nintendo_Legend"	6895
* "retrodealsUK"	6744
* "LastChanceGamer"	5904
* "GameUP247"	5580
* "RetroNuss"	5441
* "Nintendoe3E3"	5438


project_tweets02.data
* "Cammie_Whybrew"	13616
* "AskPlayStation"	13282
* "eBayShopperNews"	11578
* "VideoGamesMall"	10274
* "savetimeandmoey"	8632
* "collinschristof"	5382
* "topnewskoeln"	5297
* "Gamifive"	4829
* "Xbox_360_Gamez"	4080
* "pressebank"	3799


project_tweets03.data
* "Xbox_360_Gamez"	34137
* "XboxSupport"	14421
* "VideoGamesMall"	11835
* "Xbox_One_Reddit"	10215
* "xboxgamersdeals"	9224
* "GameUP247"	9156
* "KingsleyNewz"	9139
* "bullzyy"	7576
* "savetimeandmoey"	7176
* "giveawayxfab"	7028


project_tweets04.data
* "savetimeandmoey"	15483
* "VideoGames_Up"	13804
* "giveawaygigatop"	8878
* "tw100_1"	6538
* "videogames_pt"	6357
* "videogames_fr"	5993
* "giveawayxfab"	5886
* "DMGG_Videogames"	5295
* "ShoppeWorld"	4824
* "VideoGames_TV"	3709


Time taken to completion of the metric: 335.930534 in processor time


**Analysis**

From this we can see that (requires more analysis).

5) The nexts analysis of the data is by each hour of a day in every week (the data was gathered).

5.1) We now calculate the Top 20 words. That is how many times a single word was used in each record for each platform. This is done with the next script.

**Results**



**Analysis**




5.2) We now calculate how many records have a sentiment of "positiveness", "negativeness" or "neutralness" for each platform with the next script.

In [ ]:
import time
import sys
import sentiment_analyzer as sa

# The summary is for each hour per week. (i.e. 20 positives tweets on week 1, 31 negative tweets on week 2, ...)

if __name__ == '__main__':
    time_start = time.clock()
    sid = sa.init_classifier()
    # Clean File.
    open("sentiment_analysis_by_platform_summary.txt", 'w').close()
    files = ["project_tweets01.data", "project_tweets02.data", "project_tweets03.data", "project_tweets04.data"]
    for _file in files:
        _output_filename = sa.classify_file(_file, sid)
        # parameters for mrjob.
        # To run your job in multiple subprocesses with a few Hadoop features simulated, use -r local.
        option1 = "" #""-r"
        option2 = "" #""local"
        sys.argv = ['sentiment_amount.py', option1, option2, _output_filename]
        # Write to file in append mode.
        _fo = open("sentiment_analysis_by_platform_summary.txt", 'a')
        sys.stdout = _fo
        print _file
        execfile('sentiment_amount.py')
        print "\n"

    time_end = time.clock()

    print "Time taken to completion of the metric: {0} in processor time".format(time_end - time_start)


In [ ]:
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import json
import sys


def init_classifier():
    n_instances = 100
    subj_docs = [(sent, 'subj') for sent in subjectivity.sents(categories='subj')[:n_instances]]
    obj_docs = [(sent, 'obj') for sent in subjectivity.sents(categories='obj')[:n_instances]]

    train_subj_docs = subj_docs[:80]
    test_subj_docs = subj_docs[80:100]
    train_obj_docs = obj_docs[:80]
    test_obj_docs = obj_docs[80:100]
    training_docs = train_subj_docs + train_obj_docs
    testing_docs = test_subj_docs + test_obj_docs

    sentim_analyzer = SentimentAnalyzer()
    all_words_neg = sentim_analyzer.all_words([mark_negation(doc) for doc in training_docs])
    unigram_feats = sentim_analyzer.unigram_word_feats(all_words_neg, min_freq=4)
    sentim_analyzer.add_feat_extractor(extract_unigram_feats, unigrams=unigram_feats)
    training_set = sentim_analyzer.apply_features(training_docs)
    test_set = sentim_analyzer.apply_features(testing_docs)
    trainer = NaiveBayesClassifier.train
    classifier = sentim_analyzer.train(trainer, training_set)
    sid = SentimentIntensityAnalyzer()
    return sid


def classify_file(_file, sid):
    _data = open(_file, 'r')
    open(_file[:-5] + "_classified.data", 'w').close()
    _output = open(_file[:-5] + "_classified.data", 'a')
    for line in _data.readlines():
        record = json.loads(line)
        ss = sid.polarity_scores(record['text'].encode('utf-8'))
        if ss['compound'] < 0:
            classification = "neg"
        elif ss['compound'] == 0:
            classification = "neu"
        else:
            classification = "pos"
        try:
            record.update({'classification': classification})
            _output.write(json.dumps(record, sort_keys=True) + "\n")
        except:
            # Should not happen.
            print "WTF!"
    _data.close()
    _output.close()
    return _output.name

if __name__ == '__main__':
    pass


In [ ]:
from mrjob.job import MRJob
from mrjob.step import MRStep
from mrjob.protocol import JSONValueProtocol
import time
import itertools
import sys
from datetime import datetime
from dateutil.parser import parse

class MRWordFrequencyCount(MRJob):
    INPUT_PROTOCOL = JSONValueProtocol

    def mapper(self, _, record):
        date = parse(record['created_at'])
        week = date.isocalendar()[1]
        hour = date.hour
        yield [(week, hour, record['classification']), 1]

    def reducer(self, key, values):
        yield [key, sum(values)]

    def steps(self):
        return [MRStep(mapper=self.mapper, reducer=self.reducer)]


if __name__ == '__main__':
    #time_start = time.clock()
    MRWordFrequencyCount().run()
    #time_end = time.clock()
    #print "Time taken to completion of the metric: {0} in processor time".format(time_end - time_start)


**Results**

project_tweets03.data
[26, 0, "neg"]	17
[26, 0, "neu"]	55
[26, 0, "pos"]	29
[26, 1, "neg"]	7
[26, 1, "neu"]	44
[26, 1, "pos"]	17
[26, 10, "neg"]	15
[26, 10, "neu"]	73
[26, 10, "pos"]	33
[26, 11, "neg"]	20
[26, 11, "neu"]	68
[26, 11, "pos"]	30
[26, 12, "neg"]	20
[26, 12, "neu"]	72
[26, 12, "pos"]	32
[26, 13, "neg"]	19
[26, 13, "neu"]	81
[26, 13, "pos"]	34
[26, 14, "neg"]	27
[26, 14, "neu"]	78
[26, 14, "pos"]	41
[26, 15, "neg"]	20
[26, 15, "neu"]	90
[26, 15, "pos"]	31
[26, 16, "neg"]	14
[26, 16, "neu"]	65
[26, 16, "pos"]	34
[26, 17, "neg"]	18
[26, 17, "neu"]	72
[26, 17, "pos"]	34
[26, 18, "neg"]	16
[26, 18, "neu"]	78
[26, 18, "pos"]	42
[26, 19, "neg"]	20
[26, 19, "neu"]	80
[26, 19, "pos"]	47
[26, 2, "neg"]	15
[26, 2, "neu"]	48
[26, 2, "pos"]	24
[26, 20, "neg"]	20
[26, 20, "neu"]	76
[26, 20, "pos"]	45
[26, 21, "neg"]	36
[26, 21, "neu"]	100
[26, 21, "pos"]	45
[26, 22, "neg"]	22
[26, 22, "neu"]	84
[26, 22, "pos"]	46
[26, 23, "neg"]	21
[26, 23, "neu"]	93
[26, 23, "pos"]	33
[26, 3, "neg"]	5
[26, 3, "neu"]	48
[26, 3, "pos"]	18
[26, 4, "neg"]	12
[26, 4, "neu"]	46
[26, 4, "pos"]	15
[26, 5, "neg"]	9
[26, 5, "neu"]	35
[26, 5, "pos"]	22
[26, 6, "neg"]	11
[26, 6, "neu"]	43
[26, 6, "pos"]	22
[26, 7, "neg"]	14
[26, 7, "neu"]	31
[26, 7, "pos"]	17
[26, 8, "neg"]	11
[26, 8, "neu"]	58
[26, 8, "pos"]	20
[26, 9, "neg"]	29
[26, 9, "neu"]	66
[26, 9, "pos"]	30
[27, 0, "neg"]	42
[27, 0, "neu"]	115
[27, 0, "pos"]	71
[27, 1, "neg"]	31
[27, 1, "neu"]	142
[27, 1, "pos"]	64
[27, 10, "neg"]	301
[27, 10, "neu"]	912
[27, 10, "pos"]	549
[27, 11, "neg"]	450
[27, 11, "neu"]	1189
[27, 11, "pos"]	788
[27, 12, "neg"]	603
[27, 12, "neu"]	1064
[27, 12, "pos"]	801
[27, 13, "neg"]	588
[27, 13, "neu"]	1108
[27, 13, "pos"]	808
[27, 14, "neg"]	649
[27, 14, "neu"]	1072
[27, 14, "pos"]	989
[27, 15, "neg"]	673
[27, 15, "neu"]	1017
[27, 15, "pos"]	845
[27, 16, "neg"]	522
[27, 16, "neu"]	1069
[27, 16, "pos"]	739
[27, 17, "neg"]	424
[27, 17, "neu"]	1071
[27, 17, "pos"]	768
[27, 18, "neg"]	589
[27, 18, "neu"]	999
[27, 18, "pos"]	617
[27, 19, "neg"]	431
[27, 19, "neu"]	1038
[27, 19, "pos"]	642
[27, 2, "neg"]	41
[27, 2, "neu"]	100
[27, 2, "pos"]	49
[27, 20, "neg"]	57
[27, 20, "neu"]	165
[27, 20, "pos"]	58
[27, 21, "neg"]	45
[27, 21, "neu"]	172
[27, 21, "pos"]	71
[27, 22, "neg"]	53
[27, 22, "neu"]	142
[27, 22, "pos"]	81
[27, 23, "neg"]	46
[27, 23, "neu"]	135
[27, 23, "pos"]	68
[27, 3, "neg"]	37
[27, 3, "neu"]	97
[27, 3, "pos"]	37
[27, 4, "neg"]	31
[27, 4, "neu"]	115
[27, 4, "pos"]	55
[27, 5, "neg"]	31
[27, 5, "neu"]	106
[27, 5, "pos"]	53
[27, 6, "neg"]	171
[27, 6, "neu"]	431
[27, 6, "pos"]	205
[27, 7, "neg"]	227
[27, 7, "neu"]	644
[27, 7, "pos"]	371
[27, 8, "neg"]	221
[27, 8, "neu"]	748
[27, 8, "pos"]	430
[27, 9, "neg"]	289
[27, 9, "neu"]	961
[27, 9, "pos"]	455
[28, 0, "neg"]	322
[28, 0, "neu"]	668
[28, 0, "pos"]	363
[28, 1, "neg"]	243
[28, 1, "neu"]	662
[28, 1, "pos"]	327
[28, 10, "neg"]	284
[28, 10, "neu"]	980
[28, 10, "pos"]	451
[28, 11, "neg"]	421
[28, 11, "neu"]	1151
[28, 11, "pos"]	564
[28, 12, "neg"]	1300
[28, 12, "neu"]	1950
[28, 12, "pos"]	1181
[28, 13, "neg"]	996
[28, 13, "neu"]	2169
[28, 13, "pos"]	1291
[28, 14, "neg"]	898
[28, 14, "neu"]	2024
[28, 14, "pos"]	1239
[28, 15, "neg"]	1159
[28, 15, "neu"]	1860
[28, 15, "pos"]	1263
[28, 16, "neg"]	947
[28, 16, "neu"]	1810
[28, 16, "pos"]	1303
[28, 17, "neg"]	862
[28, 17, "neu"]	1907
[28, 17, "pos"]	1291
[28, 18, "neg"]	768
[28, 18, "neu"]	1742
[28, 18, "pos"]	1132
[28, 19, "neg"]	650
[28, 19, "neu"]	1660
[28, 19, "pos"]	938
[28, 2, "neg"]	310
[28, 2, "neu"]	552
[28, 2, "pos"]	360
[28, 20, "neg"]	548
[28, 20, "neu"]	1241
[28, 20, "pos"]	660
[28, 21, "neg"]	569
[28, 21, "neu"]	1739
[28, 21, "pos"]	909
[28, 22, "neg"]	650
[28, 22, "neu"]	1571
[28, 22, "pos"]	855
[28, 23, "neg"]	464
[28, 23, "neu"]	1407
[28, 23, "pos"]	786
[28, 3, "neg"]	251
[28, 3, "neu"]	700
[28, 3, "pos"]	297
[28, 4, "neg"]	174
[28, 4, "neu"]	641
[28, 4, "pos"]	233
[28, 5, "neg"]	189
[28, 5, "neu"]	644
[28, 5, "pos"]	324
[28, 6, "neg"]	205
[28, 6, "neu"]	580
[28, 6, "pos"]	240
[28, 7, "neg"]	212
[28, 7, "neu"]	628
[28, 7, "pos"]	333
[28, 8, "neg"]	226
[28, 8, "neu"]	743
[28, 8, "pos"]	338
[28, 9, "neg"]	277
[28, 9, "neu"]	972
[28, 9, "pos"]	471
[29, 0, "neg"]	772
[29, 0, "neu"]	2694
[29, 0, "pos"]	1391
[29, 1, "neg"]	910
[29, 1, "neu"]	2409
[29, 1, "pos"]	1283
[29, 10, "neg"]	1221
[29, 10, "neu"]	5614
[29, 10, "pos"]	1972
[29, 11, "neg"]	1179
[29, 11, "neu"]	4903
[29, 11, "pos"]	1911
[29, 12, "neg"]	1298
[29, 12, "neu"]	4887
[29, 12, "pos"]	2698
[29, 13, "neg"]	1361
[29, 13, "neu"]	4541
[29, 13, "pos"]	2658
[29, 14, "neg"]	1350
[29, 14, "neu"]	4011
[29, 14, "pos"]	2612
[29, 15, "neg"]	1411
[29, 15, "neu"]	3936
[29, 15, "pos"]	2442
[29, 16, "neg"]	1270
[29, 16, "neu"]	3834
[29, 16, "pos"]	2433
[29, 17, "neg"]	1505
[29, 17, "neu"]	3924
[29, 17, "pos"]	2392
[29, 18, "neg"]	1415
[29, 18, "neu"]	3856
[29, 18, "pos"]	2189
[29, 19, "neg"]	1661
[29, 19, "neu"]	3429
[29, 19, "pos"]	2138
[29, 2, "neg"]	805
[29, 2, "neu"]	2274
[29, 2, "pos"]	1152
[29, 20, "neg"]	1007
[29, 20, "neu"]	2442
[29, 20, "pos"]	1581
[29, 21, "neg"]	769
[29, 21, "neu"]	2438
[29, 21, "pos"]	1331
[29, 22, "neg"]	783
[29, 22, "neu"]	2500
[29, 22, "pos"]	1224
[29, 23, "neg"]	678
[29, 23, "neu"]	2295
[29, 23, "pos"]	1234
[29, 3, "neg"]	988
[29, 3, "neu"]	2326
[29, 3, "pos"]	1001
[29, 4, "neg"]	700
[29, 4, "neu"]	2210
[29, 4, "pos"]	983
[29, 5, "neg"]	790
[29, 5, "neu"]	2268
[29, 5, "pos"]	953
[29, 6, "neg"]	603
[29, 6, "neu"]	2336
[29, 6, "pos"]	920
[29, 7, "neg"]	774
[29, 7, "neu"]	2536
[29, 7, "pos"]	953
[29, 8, "neg"]	806
[29, 8, "neu"]	2658
[29, 8, "pos"]	1170
[29, 9, "neg"]	1038
[29, 9, "neu"]	5310
[29, 9, "pos"]	2035
[30, 0, "neg"]	699
[30, 0, "neu"]	2015
[30, 0, "pos"]	1111
[30, 1, "neg"]	727
[30, 1, "neu"]	2124
[30, 1, "pos"]	1152
[30, 10, "neg"]	995
[30, 10, "neu"]	3010
[30, 10, "pos"]	1585
[30, 11, "neg"]	903
[30, 11, "neu"]	2966
[30, 11, "pos"]	1748
[30, 12, "neg"]	1182
[30, 12, "neu"]	3128
[30, 12, "pos"]	2070
[30, 13, "neg"]	1074
[30, 13, "neu"]	3281
[30, 13, "pos"]	2353
[30, 14, "neg"]	1093
[30, 14, "neu"]	3287
[30, 14, "pos"]	2230
[30, 15, "neg"]	1079
[30, 15, "neu"]	3357
[30, 15, "pos"]	2517
[30, 16, "neg"]	1076
[30, 16, "neu"]	3190
[30, 16, "pos"]	2134
[30, 17, "neg"]	1056
[30, 17, "neu"]	3314
[30, 17, "pos"]	2388
[30, 18, "neg"]	1141
[30, 18, "neu"]	2779
[30, 18, "pos"]	1950
[30, 19, "neg"]	1088
[30, 19, "neu"]	2747
[30, 19, "pos"]	1633
[30, 2, "neg"]	527
[30, 2, "neu"]	1841
[30, 2, "pos"]	974
[30, 20, "neg"]	719
[30, 20, "neu"]	1762
[30, 20, "pos"]	1114
[30, 21, "neg"]	685
[30, 21, "neu"]	1906
[30, 21, "pos"]	1110
[30, 22, "neg"]	625
[30, 22, "neu"]	1658
[30, 22, "pos"]	959
[30, 23, "neg"]	537
[30, 23, "neu"]	1647
[30, 23, "pos"]	878
[30, 3, "neg"]	573
[30, 3, "neu"]	1766
[30, 3, "pos"]	799
[30, 4, "neg"]	502
[30, 4, "neu"]	1946
[30, 4, "pos"]	889
[30, 5, "neg"]	457
[30, 5, "neu"]	2113
[30, 5, "pos"]	828
[30, 6, "neg"]	584
[30, 6, "neu"]	1827
[30, 6, "pos"]	831
[30, 7, "neg"]	636
[30, 7, "neu"]	2173
[30, 7, "pos"]	975
[30, 8, "neg"]	634
[30, 8, "neu"]	2036
[30, 8, "pos"]	944
[30, 9, "neg"]	723
[30, 9, "neu"]	2816
[30, 9, "pos"]	1504
[31, 0, "neg"]	739
[31, 0, "neu"]	2179
[31, 0, "pos"]	1590
[31, 1, "neg"]	690
[31, 1, "neu"]	1983
[31, 1, "pos"]	1459
[31, 10, "neg"]	1013
[31, 10, "neu"]	3820
[31, 10, "pos"]	2154
[31, 11, "neg"]	1193
[31, 11, "neu"]	3894
[31, 11, "pos"]	2392
[31, 12, "neg"]	1421
[31, 12, "neu"]	4843
[31, 12, "pos"]	3275
[31, 13, "neg"]	1328
[31, 13, "neu"]	4426
[31, 13, "pos"]	3016
[31, 14, "neg"]	1751
[31, 14, "neu"]	5656
[31, 14, "pos"]	3597
[31, 15, "neg"]	1748
[31, 15, "neu"]	5294
[31, 15, "pos"]	3836
[31, 16, "neg"]	1884
[31, 16, "neu"]	5037
[31, 16, "pos"]	3621
[31, 17, "neg"]	2100
[31, 17, "neu"]	5348
[31, 17, "pos"]	3999
[31, 18, "neg"]	1848
[31, 18, "neu"]	4765
[31, 18, "pos"]	3599
[31, 19, "neg"]	1644
[31, 19, "neu"]	4338
[31, 19, "pos"]	3014
[31, 2, "neg"]	714
[31, 2, "neu"]	1893
[31, 2, "pos"]	1347
[31, 20, "neg"]	1437
[31, 20, "neu"]	3460
[31, 20, "pos"]	2459
[31, 21, "neg"]	1240
[31, 21, "neu"]	3539
[31, 21, "pos"]	2608
[31, 22, "neg"]	972
[31, 22, "neu"]	3020
[31, 22, "pos"]	2141
[31, 23, "neg"]	1002
[31, 23, "neu"]	3061
[31, 23, "pos"]	2225
[31, 3, "neg"]	562
[31, 3, "neu"]	2049
[31, 3, "pos"]	1039
[31, 4, "neg"]	630
[31, 4, "neu"]	2115
[31, 4, "pos"]	1256
[31, 5, "neg"]	540
[31, 5, "neu"]	2159
[31, 5, "pos"]	1037
[31, 6, "neg"]	511
[31, 6, "neu"]	2240
[31, 6, "pos"]	1062
[31, 7, "neg"]	662
[31, 7, "neu"]	2407
[31, 7, "pos"]	1378
[31, 8, "neg"]	781
[31, 8, "neu"]	2555
[31, 8, "pos"]	1328
[31, 9, "neg"]	1018
[31, 9, "neu"]	3876
[31, 9, "pos"]	3028
[32, 0, "neg"]	905
[32, 0, "neu"]	3050
[32, 0, "pos"]	1414
[32, 1, "neg"]	714
[32, 1, "neu"]	2254
[32, 1, "pos"]	1303
[32, 10, "neg"]	993
[32, 10, "neu"]	3110
[32, 10, "pos"]	1717
[32, 11, "neg"]	1102
[32, 11, "neu"]	3719
[32, 11, "pos"]	1899
[32, 12, "neg"]	1658
[32, 12, "neu"]	4027
[32, 12, "pos"]	2499
[32, 13, "neg"]	1414
[32, 13, "neu"]	4049
[32, 13, "pos"]	2541
[32, 14, "neg"]	1510
[32, 14, "neu"]	3797
[32, 14, "pos"]	2448
[32, 15, "neg"]	1499
[32, 15, "neu"]	3585
[32, 15, "pos"]	2604
[32, 16, "neg"]	1357
[32, 16, "neu"]	3302
[32, 16, "pos"]	2472
[32, 17, "neg"]	1480
[32, 17, "neu"]	3509
[32, 17, "pos"]	2288
[32, 18, "neg"]	1281
[32, 18, "neu"]	3565
[32, 18, "pos"]	2304
[32, 19, "neg"]	1212
[32, 19, "neu"]	3361
[32, 19, "pos"]	2191
[32, 2, "neg"]	657
[32, 2, "neu"]	2166
[32, 2, "pos"]	1234
[32, 20, "neg"]	845
[32, 20, "neu"]	2468
[32, 20, "pos"]	1467
[32, 21, "neg"]	790
[32, 21, "neu"]	2337
[32, 21, "pos"]	1473
[32, 22, "neg"]	809
[32, 22, "neu"]	2223
[32, 22, "pos"]	1401
[32, 23, "neg"]	786
[32, 23, "neu"]	2228
[32, 23, "pos"]	1213
[32, 3, "neg"]	632
[32, 3, "neu"]	1935
[32, 3, "pos"]	1052
[32, 4, "neg"]	749
[32, 4, "neu"]	2090
[32, 4, "pos"]	978
[32, 5, "neg"]	674
[32, 5, "neu"]	2087
[32, 5, "pos"]	1064
[32, 6, "neg"]	695
[32, 6, "neu"]	2311
[32, 6, "pos"]	1029
[32, 7, "neg"]	698
[32, 7, "neu"]	2391
[32, 7, "pos"]	1166
[32, 8, "neg"]	816
[32, 8, "neu"]	2976
[32, 8, "pos"]	1273
[32, 9, "neg"]	887
[32, 9, "neu"]	3076
[32, 9, "pos"]	1385
[33, 0, "neg"]	820
[33, 0, "neu"]	2064
[33, 0, "pos"]	1228
[33, 1, "neg"]	778
[33, 1, "neu"]	2022
[33, 1, "pos"]	1208
[33, 10, "neg"]	1111
[33, 10, "neu"]	3543
[33, 10, "pos"]	1827
[33, 11, "neg"]	1093
[33, 11, "neu"]	3684
[33, 11, "pos"]	1909
[33, 12, "neg"]	1118
[33, 12, "neu"]	3832
[33, 12, "pos"]	2149
[33, 13, "neg"]	1470
[33, 13, "neu"]	5472
[33, 13, "pos"]	3198
[33, 14, "neg"]	2247
[33, 14, "neu"]	4915
[33, 14, "pos"]	3209
[33, 15, "neg"]	1982
[33, 15, "neu"]	4904
[33, 15, "pos"]	2958
[33, 16, "neg"]	1703
[33, 16, "neu"]	5029
[33, 16, "pos"]	2988
[33, 17, "neg"]	1600
[33, 17, "neu"]	4105
[33, 17, "pos"]	2801
[33, 18, "neg"]	1489
[33, 18, "neu"]	4036
[33, 18, "pos"]	2940
[33, 19, "neg"]	1453
[33, 19, "neu"]	4087
[33, 19, "pos"]	2506
[33, 2, "neg"]	580
[33, 2, "neu"]	1694
[33, 2, "pos"]	1073
[33, 20, "neg"]	1229
[33, 20, "neu"]	3653
[33, 20, "pos"]	2584
[33, 21, "neg"]	983
[33, 21, "neu"]	2740
[33, 21, "pos"]	1705
[33, 22, "neg"]	893
[33, 22, "neu"]	2614
[33, 22, "pos"]	1705
[33, 23, "neg"]	822
[33, 23, "neu"]	2423
[33, 23, "pos"]	1392
[33, 3, "neg"]	555
[33, 3, "neu"]	1818
[33, 3, "pos"]	988
[33, 4, "neg"]	613
[33, 4, "neu"]	2123
[33, 4, "pos"]	998
[33, 5, "neg"]	684
[33, 5, "neu"]	2548
[33, 5, "pos"]	1066
[33, 6, "neg"]	704
[33, 6, "neu"]	2611
[33, 6, "pos"]	1079
[33, 7, "neg"]	695
[33, 7, "neu"]	2632
[33, 7, "pos"]	1071
[33, 8, "neg"]	840
[33, 8, "neu"]	2557
[33, 8, "pos"]	1339
[33, 9, "neg"]	951
[33, 9, "neu"]	3200
[33, 9, "pos"]	1697
[34, 0, "neg"]	792
[34, 0, "neu"]	2670
[34, 0, "pos"]	1745
[34, 1, "neg"]	664
[34, 1, "neu"]	2443
[34, 1, "pos"]	1378
[34, 10, "neg"]	738
[34, 10, "neu"]	2918
[34, 10, "pos"]	1578
[34, 11, "neg"]	946
[34, 11, "neu"]	3220
[34, 11, "pos"]	1657
[34, 12, "neg"]	1123
[34, 12, "neu"]	3683
[34, 12, "pos"]	2223
[34, 13, "neg"]	1497
[34, 13, "neu"]	5052
[34, 13, "pos"]	3128
[34, 14, "neg"]	1376
[34, 14, "neu"]	5067
[34, 14, "pos"]	3449
[34, 15, "neg"]	1261
[34, 15, "neu"]	4954
[34, 15, "pos"]	3329
[34, 16, "neg"]	1307
[34, 16, "neu"]	4720
[34, 16, "pos"]	3365
[34, 17, "neg"]	1270
[34, 17, "neu"]	4415
[34, 17, "pos"]	3224
[34, 18, "neg"]	1379
[34, 18, "neu"]	4263
[34, 18, "pos"]	3159
[34, 19, "neg"]	1320
[34, 19, "neu"]	3975
[34, 19, "pos"]	2983
[34, 2, "neg"]	563
[34, 2, "neu"]	2030
[34, 2, "pos"]	1263
[34, 20, "neg"]	1303
[34, 20, "neu"]	4321
[34, 20, "pos"]	2673
[34, 21, "neg"]	982
[34, 21, "neu"]	3121
[34, 21, "pos"]	1847
[34, 22, "neg"]	909
[34, 22, "neu"]	3266
[34, 22, "pos"]	1826
[34, 23, "neg"]	1109
[34, 23, "neu"]	2734
[34, 23, "pos"]	2365
[34, 3, "neg"]	471
[34, 3, "neu"]	1786
[34, 3, "pos"]	1078
[34, 4, "neg"]	470
[34, 4, "neu"]	1949
[34, 4, "pos"]	970
[34, 5, "neg"]	440
[34, 5, "neu"]	1898
[34, 5, "pos"]	894
[34, 6, "neg"]	455
[34, 6, "neu"]	2095
[34, 6, "pos"]	919
[34, 7, "neg"]	801
[34, 7, "neu"]	2508
[34, 7, "pos"]	1509
[34, 8, "neg"]	676
[34, 8, "neu"]	2596
[34, 8, "pos"]	1388
[34, 9, "neg"]	772
[34, 9, "neu"]	2825
[34, 9, "pos"]	1296
[35, 0, "neg"]	705
[35, 0, "neu"]	2278
[35, 0, "pos"]	1306
[35, 1, "neg"]	655
[35, 1, "neu"]	2098
[35, 1, "pos"]	1195
[35, 10, "neg"]	1035
[35, 10, "neu"]	3402
[35, 10, "pos"]	2000
[35, 11, "neg"]	1257
[35, 11, "neu"]	3519
[35, 11, "pos"]	2017
[35, 12, "neg"]	1335
[35, 12, "neu"]	3775
[35, 12, "pos"]	2807
[35, 13, "neg"]	1482
[35, 13, "neu"]	4291
[35, 13, "pos"]	3076
[35, 14, "neg"]	1318
[35, 14, "neu"]	3996
[35, 14, "pos"]	2964
[35, 15, "neg"]	1406
[35, 15, "neu"]	4078
[35, 15, "pos"]	3298
[35, 16, "neg"]	1467
[35, 16, "neu"]	4081
[35, 16, "pos"]	3123
[35, 17, "neg"]	1408
[35, 17, "neu"]	4017
[35, 17, "pos"]	3097
[35, 18, "neg"]	1440
[35, 18, "neu"]	3990
[35, 18, "pos"]	2852
[35, 19, "neg"]	1389
[35, 19, "neu"]	3777
[35, 19, "pos"]	2742
[35, 2, "neg"]	722
[35, 2, "neu"]	2238
[35, 2, "pos"]	1279
[35, 20, "neg"]	1474
[35, 20, "neu"]	4041
[35, 20, "pos"]	2594
[35, 21, "neg"]	1108
[35, 21, "neu"]	3064
[35, 21, "pos"]	1697
[35, 22, "neg"]	894
[35, 22, "neu"]	2607
[35, 22, "pos"]	1603
[35, 23, "neg"]	1003
[35, 23, "neu"]	2721
[35, 23, "pos"]	1628
[35, 3, "neg"]	820
[35, 3, "neu"]	2423
[35, 3, "pos"]	1214
[35, 4, "neg"]	772
[35, 4, "neu"]	2380
[35, 4, "pos"]	1139
[35, 5, "neg"]	806
[35, 5, "neu"]	2581
[35, 5, "pos"]	1051
[35, 6, "neg"]	743
[35, 6, "neu"]	2606
[35, 6, "pos"]	1005
[35, 7, "neg"]	863
[35, 7, "neu"]	2611
[35, 7, "pos"]	1124
[35, 8, "neg"]	993
[35, 8, "neu"]	2817
[35, 8, "pos"]	1324
[35, 9, "neg"]	1034
[35, 9, "neu"]	2951
[35, 9, "pos"]	1657
[36, 0, "neg"]	866
[36, 0, "neu"]	2464
[36, 0, "pos"]	1483
[36, 1, "neg"]	737
[36, 1, "neu"]	2436
[36, 1, "pos"]	1489
[36, 10, "neg"]	879
[36, 10, "neu"]	3006
[36, 10, "pos"]	1487
[36, 11, "neg"]	983
[36, 11, "neu"]	3519
[36, 11, "pos"]	1896
[36, 12, "neg"]	1092
[36, 12, "neu"]	3694
[36, 12, "pos"]	2512
[36, 13, "neg"]	1263
[36, 13, "neu"]	3945
[36, 13, "pos"]	2546
[36, 14, "neg"]	1307
[36, 14, "neu"]	4135
[36, 14, "pos"]	2762
[36, 15, "neg"]	1319
[36, 15, "neu"]	3720
[36, 15, "pos"]	2589
[36, 16, "neg"]	1893
[36, 16, "neu"]	5517
[36, 16, "pos"]	3726
[36, 17, "neg"]	2211
[36, 17, "neu"]	5555
[36, 17, "pos"]	4276
[36, 18, "neg"]	1819
[36, 18, "neu"]	4988
[36, 18, "pos"]	3310
[36, 19, "neg"]	1678
[36, 19, "neu"]	4561
[36, 19, "pos"]	2990
[36, 2, "neg"]	686
[36, 2, "neu"]	2197
[36, 2, "pos"]	1348
[36, 20, "neg"]	1443
[36, 20, "neu"]	3802
[36, 20, "pos"]	2788
[36, 21, "neg"]	1107
[36, 21, "neu"]	2620
[36, 21, "pos"]	1901
[36, 22, "neg"]	1292
[36, 22, "neu"]	3553
[36, 22, "pos"]	2389
[36, 23, "neg"]	1357
[36, 23, "neu"]	4081
[36, 23, "pos"]	2320
[36, 3, "neg"]	713
[36, 3, "neu"]	1824
[36, 3, "pos"]	1061
[36, 4, "neg"]	586
[36, 4, "neu"]	1981
[36, 4, "pos"]	1004
[36, 5, "neg"]	537
[36, 5, "neu"]	2035
[36, 5, "pos"]	1009
[36, 6, "neg"]	543
[36, 6, "neu"]	2405
[36, 6, "pos"]	1035
[36, 7, "neg"]	531
[36, 7, "neu"]	2165
[36, 7, "pos"]	1010
[36, 8, "neg"]	612
[36, 8, "neu"]	2554
[36, 8, "pos"]	1275
[36, 9, "neg"]	803
[36, 9, "neu"]	2998
[36, 9, "pos"]	1418
[37, 0, "neg"]	693
[37, 0, "neu"]	2681
[37, 0, "pos"]	1333
[37, 1, "neg"]	715
[37, 1, "neu"]	2518
[37, 1, "pos"]	1363
[37, 10, "neg"]	2795
[37, 10, "neu"]	3358
[37, 10, "pos"]	1738
[37, 11, "neg"]	1077
[37, 11, "neu"]	2057
[37, 11, "pos"]	1337
[37, 12, "neg"]	1569
[37, 12, "neu"]	2144
[37, 12, "pos"]	1310
[37, 13, "neg"]	1445
[37, 13, "neu"]	2961
[37, 13, "pos"]	2222
[37, 14, "neg"]	1693
[37, 14, "neu"]	4065
[37, 14, "pos"]	2594
[37, 15, "neg"]	1875
[37, 15, "neu"]	4138
[37, 15, "pos"]	4316
[37, 16, "neg"]	922
[37, 16, "neu"]	2119
[37, 16, "pos"]	1412
[37, 17, "neg"]	1084
[37, 17, "neu"]	1929
[37, 17, "pos"]	1421
[37, 18, "neg"]	721
[37, 18, "neu"]	2096
[37, 18, "pos"]	1440
[37, 19, "neg"]	755
[37, 19, "neu"]	2024
[37, 19, "pos"]	1145
[37, 2, "neg"]	807
[37, 2, "neu"]	3263
[37, 2, "pos"]	1891
[37, 20, "neg"]	810
[37, 20, "neu"]	1963
[37, 20, "pos"]	1236
[37, 21, "neg"]	416
[37, 21, "neu"]	1078
[37, 21, "pos"]	615
[37, 22, "neg"]	494
[37, 22, "neu"]	1032
[37, 22, "pos"]	673
[37, 23, "neg"]	347
[37, 23, "neu"]	975
[37, 23, "pos"]	569
[37, 3, "neg"]	629
[37, 3, "neu"]	3078
[37, 3, "pos"]	1226
[37, 4, "neg"]	430
[37, 4, "neu"]	1974
[37, 4, "pos"]	851
[37, 5, "neg"]	639
[37, 5, "neu"]	3035
[37, 5, "pos"]	1078
[37, 6, "neg"]	647
[37, 6, "neu"]	3500
[37, 6, "pos"]	1081
[37, 7, "neg"]	673
[37, 7, "neu"]	2533
[37, 7, "pos"]	1292
[37, 8, "neg"]	673
[37, 8, "neu"]	3107
[37, 8, "pos"]	1095
[37, 9, "neg"]	843
[37, 9, "neu"]	3106
[37, 9, "pos"]	1285
[38, 0, "neg"]	276
[38, 0, "neu"]	904
[38, 0, "pos"]	437
[38, 1, "neg"]	233
[38, 1, "neu"]	640
[38, 1, "pos"]	370
[38, 10, "neg"]	588
[38, 10, "neu"]	1627
[38, 10, "pos"]	805
[38, 11, "neg"]	561
[38, 11, "neu"]	1836
[38, 11, "pos"]	1010
[38, 12, "neg"]	619
[38, 12, "neu"]	2164
[38, 12, "pos"]	1075
[38, 13, "neg"]	667
[38, 13, "neu"]	2378
[38, 13, "pos"]	1725
[38, 14, "neg"]	729
[38, 14, "neu"]	2276
[38, 14, "pos"]	1523
[38, 15, "neg"]	726
[38, 15, "neu"]	2318
[38, 15, "pos"]	1417
[38, 16, "neg"]	608
[38, 16, "neu"]	2137
[38, 16, "pos"]	1483
[38, 17, "neg"]	682
[38, 17, "neu"]	2082
[38, 17, "pos"]	1412
[38, 18, "neg"]	816
[38, 18, "neu"]	2207
[38, 18, "pos"]	1569
[38, 19, "neg"]	655
[38, 19, "neu"]	1905
[38, 19, "pos"]	1371
[38, 2, "neg"]	156
[38, 2, "neu"]	578
[38, 2, "pos"]	312
[38, 20, "neg"]	694
[38, 20, "neu"]	1945
[38, 20, "pos"]	1312
[38, 21, "neg"]	277
[38, 21, "neu"]	1178
[38, 21, "pos"]	589
[38, 22, "neg"]	332
[38, 22, "neu"]	1237
[38, 22, "pos"]	562
[38, 23, "neg"]	264
[38, 23, "neu"]	1043
[38, 23, "pos"]	508
[38, 3, "neg"]	154
[38, 3, "neu"]	612
[38, 3, "pos"]	338
[38, 4, "neg"]	226
[38, 4, "neu"]	1155
[38, 4, "pos"]	464
[38, 5, "neg"]	284
[38, 5, "neu"]	1148
[38, 5, "pos"]	521
[38, 6, "neg"]	333
[38, 6, "neu"]	1187
[38, 6, "pos"]	525
[38, 7, "neg"]	311
[38, 7, "neu"]	1316
[38, 7, "pos"]	654
[38, 8, "neg"]	387
[38, 8, "neu"]	1667
[38, 8, "pos"]	648
[38, 9, "neg"]	471
[38, 9, "neu"]	1636
[38, 9, "pos"]	795
[39, 0, "neg"]	2326
[39, 0, "neu"]	7399
[39, 0, "pos"]	5179
[39, 1, "neg"]	2067
[39, 1, "neu"]	6696
[39, 1, "pos"]	4169
[39, 10, "neg"]	1597
[39, 10, "neu"]	5144
[39, 10, "pos"]	2742
[39, 11, "neg"]	1217
[39, 11, "neu"]	3987
[39, 11, "pos"]	2103
[39, 12, "neg"]	1183
[39, 12, "neu"]	4645
[39, 12, "pos"]	2198
[39, 13, "neg"]	1868
[39, 13, "neu"]	5686
[39, 13, "pos"]	3343
[39, 14, "neg"]	2852
[39, 14, "neu"]	8082
[39, 14, "pos"]	5790
[39, 15, "neg"]	3480
[39, 15, "neu"]	11591
[39, 15, "pos"]	7723
[39, 16, "neg"]	4354
[39, 16, "neu"]	11700
[39, 16, "pos"]	8426
[39, 17, "neg"]	2695
[39, 17, "neu"]	9528
[39, 17, "pos"]	6182
[39, 18, "neg"]	3423
[39, 18, "neu"]	9307
[39, 18, "pos"]	8283
[39, 19, "neg"]	2586
[39, 19, "neu"]	8142
[39, 19, "pos"]	6658
[39, 2, "neg"]	1610
[39, 2, "neu"]	5118
[39, 2, "pos"]	3735
[39, 20, "neg"]	2635
[39, 20, "neu"]	7344
[39, 20, "pos"]	6463
[39, 21, "neg"]	1711
[39, 21, "neu"]	5789
[39, 21, "pos"]	4307
[39, 22, "neg"]	2186
[39, 22, "neu"]	7004
[39, 22, "pos"]	5223
[39, 23, "neg"]	3021
[39, 23, "neu"]	9042
[39, 23, "pos"]	6144
[39, 3, "neg"]	1601
[39, 3, "neu"]	5029
[39, 3, "pos"]	3385
[39, 4, "neg"]	1462
[39, 4, "neu"]	4714
[39, 4, "pos"]	3366
[39, 5, "neg"]	1557
[39, 5, "neu"]	4539
[39, 5, "pos"]	3341
[39, 6, "neg"]	1397
[39, 6, "neu"]	4527
[39, 6, "pos"]	2771
[39, 7, "neg"]	1357
[39, 7, "neu"]	4651
[39, 7, "pos"]	2768
[39, 8, "neg"]	1428
[39, 8, "neu"]	5226
[39, 8, "pos"]	2915
[39, 9, "neg"]	1755
[39, 9, "neu"]	5841
[39, 9, "pos"]	3237
[40, 0, "neg"]	1342
[40, 0, "neu"]	3379
[40, 0, "pos"]	2878
[40, 1, "neg"]	1777
[40, 1, "neu"]	4384
[40, 1, "pos"]	3861
[40, 10, "neg"]	695
[40, 10, "neu"]	2083
[40, 10, "pos"]	1444
[40, 11, "neg"]	354
[40, 11, "neu"]	1172
[40, 11, "pos"]	726
[40, 12, "neg"]	328
[40, 12, "neu"]	1055
[40, 12, "pos"]	739
[40, 13, "neg"]	408
[40, 13, "neu"]	1062
[40, 13, "pos"]	773
[40, 14, "neg"]	946
[40, 14, "neu"]	2355
[40, 14, "pos"]	1899
[40, 15, "neg"]	1182
[40, 15, "neu"]	3393
[40, 15, "pos"]	3031
[40, 16, "neg"]	2235
[40, 16, "neu"]	7831
[40, 16, "pos"]	6940
[40, 17, "neg"]	2597
[40, 17, "neu"]	7836
[40, 17, "pos"]	6660
[40, 18, "neg"]	2028
[40, 18, "neu"]	5223
[40, 18, "pos"]	4275
[40, 19, "neg"]	1322
[40, 19, "neu"]	3351
[40, 19, "pos"]	3119
[40, 2, "neg"]	2005
[40, 2, "neu"]	5405
[40, 2, "pos"]	4358
[40, 20, "neg"]	1587
[40, 20, "neu"]	3839
[40, 20, "pos"]	3389
[40, 21, "neg"]	1521
[40, 21, "neu"]	3616
[40, 21, "pos"]	2764
[40, 22, "neg"]	1406
[40, 22, "neu"]	4430
[40, 22, "pos"]	3574
[40, 23, "neg"]	993
[40, 23, "neu"]	2637
[40, 23, "pos"]	2332
[40, 3, "neg"]	2485
[40, 3, "neu"]	6258
[40, 3, "pos"]	6244
[40, 4, "neg"]	1493
[40, 4, "neu"]	3996
[40, 4, "pos"]	3812
[40, 5, "neg"]	1107
[40, 5, "neu"]	2941
[40, 5, "pos"]	2824
[40, 6, "neg"]	934
[40, 6, "neu"]	2511
[40, 6, "pos"]	2652
[40, 7, "neg"]	1187
[40, 7, "neu"]	2457
[40, 7, "pos"]	2048
[40, 8, "neg"]	830
[40, 8, "neu"]	3191
[40, 8, "pos"]	1822
[40, 9, "neg"]	973
[40, 9, "neu"]	3938
[40, 9, "pos"]	2345
[41, 0, "neg"]	1740
[41, 0, "neu"]	4784
[41, 0, "pos"]	3683
[41, 1, "neg"]	1663
[41, 1, "neu"]	4056
[41, 1, "pos"]	2956
[41, 10, "neg"]	901
[41, 10, "neu"]	3186
[41, 10, "pos"]	1736
[41, 11, "neg"]	542
[41, 11, "neu"]	2237
[41, 11, "pos"]	1438
[41, 12, "neg"]	637
[41, 12, "neu"]	2345
[41, 12, "pos"]	1690
[41, 13, "neg"]	1234
[41, 13, "neu"]	3824
[41, 13, "pos"]	2890
[41, 14, "neg"]	2126
[41, 14, "neu"]	6909
[41, 14, "pos"]	4517
[41, 15, "neg"]	2281
[41, 15, "neu"]	6869
[41, 15, "pos"]	4620
[41, 16, "neg"]	3156
[41, 16, "neu"]	9840
[41, 16, "pos"]	9220
[41, 17, "neg"]	2223
[41, 17, "neu"]	6059
[41, 17, "pos"]	5606
[41, 18, "neg"]	1728
[41, 18, "neu"]	4393
[41, 18, "pos"]	6090
[41, 19, "neg"]	1837
[41, 19, "neu"]	5085
[41, 19, "pos"]	5777
[41, 2, "neg"]	1876
[41, 2, "neu"]	5367
[41, 2, "pos"]	4021
[41, 20, "neg"]	1767
[41, 20, "neu"]	5181
[41, 20, "pos"]	4482
[41, 21, "neg"]	1300
[41, 21, "neu"]	4005
[41, 21, "pos"]	3678
[41, 22, "neg"]	2437
[41, 22, "neu"]	7445
[41, 22, "pos"]	6835
[41, 23, "neg"]	2396
[41, 23, "neu"]	6534
[41, 23, "pos"]	5078
[41, 3, "neg"]	1715
[41, 3, "neu"]	5009
[41, 3, "pos"]	4015
[41, 4, "neg"]	1516
[41, 4, "neu"]	4395
[41, 4, "pos"]	3566
[41, 5, "neg"]	1420
[41, 5, "neu"]	4237
[41, 5, "pos"]	2997
[41, 6, "neg"]	1274
[41, 6, "neu"]	3997
[41, 6, "pos"]	2611
[41, 7, "neg"]	1210
[41, 7, "neu"]	3548
[41, 7, "pos"]	2219
[41, 8, "neg"]	1237
[41, 8, "neu"]	4068
[41, 8, "pos"]	2515
[41, 9, "neg"]	1186
[41, 9, "neu"]	4989
[41, 9, "pos"]	2816
[42, 0, "neg"]	1826
[42, 0, "neu"]	3386
[42, 0, "pos"]	3175
[42, 1, "neg"]	1922
[42, 1, "neu"]	2934
[42, 1, "pos"]	2696
[42, 13, "neg"]	321
[42, 13, "neu"]	643
[42, 13, "pos"]	405
[42, 14, "neg"]	553
[42, 14, "neu"]	1031
[42, 14, "pos"]	636
[42, 15, "neg"]	70
[42, 15, "neu"]	226
[42, 15, "pos"]	231
[42, 16, "neg"]	768
[42, 16, "neu"]	1365
[42, 16, "pos"]	981
[42, 17, "neg"]	1080
[42, 17, "neu"]	2042
[42, 17, "pos"]	1779
[42, 18, "neg"]	346
[42, 18, "neu"]	1131
[42, 18, "pos"]	797
[42, 2, "neg"]	1768
[42, 2, "neu"]	3631
[42, 2, "pos"]	2634
[42, 3, "neg"]	639
[42, 3, "neu"]	1671
[42, 3, "pos"]	1543


project_tweets04.data
[25, 23, "neu"]	1
[26, 0, "neg"]	61
[26, 0, "neu"]	175
[26, 0, "pos"]	119
[26, 1, "neg"]	61
[26, 1, "neu"]	168
[26, 1, "pos"]	98
[26, 10, "neg"]	73
[26, 10, "neu"]	279
[26, 10, "pos"]	153
[26, 11, "neg"]	88
[26, 11, "neu"]	238
[26, 11, "pos"]	161
[26, 12, "neg"]	89
[26, 12, "neu"]	350
[26, 12, "pos"]	200
[26, 13, "neg"]	76
[26, 13, "neu"]	290
[26, 13, "pos"]	176
[26, 14, "neg"]	78
[26, 14, "neu"]	267
[26, 14, "pos"]	173
[26, 15, "neg"]	94
[26, 15, "neu"]	342
[26, 15, "pos"]	188
[26, 16, "neg"]	93
[26, 16, "neu"]	309
[26, 16, "pos"]	233
[26, 17, "neg"]	98
[26, 17, "neu"]	284
[26, 17, "pos"]	210
[26, 18, "neg"]	98
[26, 18, "neu"]	260
[26, 18, "pos"]	170
[26, 19, "neg"]	61
[26, 19, "neu"]	240
[26, 19, "pos"]	145
[26, 2, "neg"]	51
[26, 2, "neu"]	168
[26, 2, "pos"]	130
[26, 20, "neg"]	96
[26, 20, "neu"]	300
[26, 20, "pos"]	190
[26, 21, "neg"]	103
[26, 21, "neu"]	384
[26, 21, "pos"]	173
[26, 22, "neg"]	95
[26, 22, "neu"]	276
[26, 22, "pos"]	181
[26, 23, "neg"]	102
[26, 23, "neu"]	331
[26, 23, "pos"]	180
[26, 3, "neg"]	51
[26, 3, "neu"]	199
[26, 3, "pos"]	115
[26, 4, "neg"]	56
[26, 4, "neu"]	236
[26, 4, "pos"]	114
[26, 5, "neg"]	42
[26, 5, "neu"]	342
[26, 5, "pos"]	124
[26, 6, "neg"]	63
[26, 6, "neu"]	198
[26, 6, "pos"]	89
[26, 7, "neg"]	59
[26, 7, "neu"]	204
[26, 7, "pos"]	104
[26, 8, "neg"]	69
[26, 8, "neu"]	238
[26, 8, "pos"]	122
[26, 9, "neg"]	71
[26, 9, "neu"]	223
[26, 9, "pos"]	159
[27, 0, "neg"]	153
[27, 0, "neu"]	493
[27, 0, "pos"]	291
[27, 1, "neg"]	157
[27, 1, "neu"]	426
[27, 1, "pos"]	288
[27, 10, "neg"]	239
[27, 10, "neu"]	991
[27, 10, "pos"]	548
[27, 11, "neg"]	262
[27, 11, "neu"]	991
[27, 11, "pos"]	536
[27, 12, "neg"]	284
[27, 12, "neu"]	934
[27, 12, "pos"]	593
[27, 13, "neg"]	289
[27, 13, "neu"]	925
[27, 13, "pos"]	868
[27, 14, "neg"]	256
[27, 14, "neu"]	852
[27, 14, "pos"]	615
[27, 15, "neg"]	285
[27, 15, "neu"]	980
[27, 15, "pos"]	520
[27, 16, "neg"]	253
[27, 16, "neu"]	916
[27, 16, "pos"]	557
[27, 17, "neg"]	251
[27, 17, "neu"]	766
[27, 17, "pos"]	549
[27, 18, "neg"]	196
[27, 18, "neu"]	732
[27, 18, "pos"]	455
[27, 19, "neg"]	247
[27, 19, "neu"]	705
[27, 19, "pos"]	431
[27, 2, "neg"]	142
[27, 2, "neu"]	484
[27, 2, "pos"]	283
[27, 20, "neg"]	190
[27, 20, "neu"]	701
[27, 20, "pos"]	322
[27, 21, "neg"]	199
[27, 21, "neu"]	611
[27, 21, "pos"]	336
[27, 22, "neg"]	184
[27, 22, "neu"]	567
[27, 22, "pos"]	401
[27, 23, "neg"]	178
[27, 23, "neu"]	535
[27, 23, "pos"]	322
[27, 3, "neg"]	125
[27, 3, "neu"]	490
[27, 3, "pos"]	238
[27, 4, "neg"]	170
[27, 4, "neu"]	648
[27, 4, "pos"]	349
[27, 5, "neg"]	169
[27, 5, "neu"]	602
[27, 5, "pos"]	281
[27, 6, "neg"]	170
[27, 6, "neu"]	593
[27, 6, "pos"]	315
[27, 7, "neg"]	202
[27, 7, "neu"]	730
[27, 7, "pos"]	391
[27, 8, "neg"]	178
[27, 8, "neu"]	734
[27, 8, "pos"]	378
[27, 9, "neg"]	258
[27, 9, "neu"]	812
[27, 9, "pos"]	430
[28, 0, "neg"]	129
[28, 0, "neu"]	519
[28, 0, "pos"]	273
[28, 1, "neg"]	135
[28, 1, "neu"]	393
[28, 1, "pos"]	227
[28, 10, "neg"]	256
[28, 10, "neu"]	1154
[28, 10, "pos"]	567
[28, 11, "neg"]	297
[28, 11, "neu"]	1126
[28, 11, "pos"]	573
[28, 12, "neg"]	326
[28, 12, "neu"]	1196
[28, 12, "pos"]	690
[28, 13, "neg"]	406
[28, 13, "neu"]	1072
[28, 13, "pos"]	629
[28, 14, "neg"]	301
[28, 14, "neu"]	1043
[28, 14, "pos"]	625
[28, 15, "neg"]	298
[28, 15, "neu"]	1155
[28, 15, "pos"]	647
[28, 16, "neg"]	263
[28, 16, "neu"]	876
[28, 16, "pos"]	602
[28, 17, "neg"]	242
[28, 17, "neu"]	841
[28, 17, "pos"]	506
[28, 18, "neg"]	221
[28, 18, "neu"]	861
[28, 18, "pos"]	505
[28, 19, "neg"]	208
[28, 19, "neu"]	715
[28, 19, "pos"]	375
[28, 2, "neg"]	150
[28, 2, "neu"]	487
[28, 2, "pos"]	265
[28, 20, "neg"]	180
[28, 20, "neu"]	591
[28, 20, "pos"]	329
[28, 21, "neg"]	165
[28, 21, "neu"]	560
[28, 21, "pos"]	427
[28, 22, "neg"]	145
[28, 22, "neu"]	554
[28, 22, "pos"]	422
[28, 23, "neg"]	164
[28, 23, "neu"]	604
[28, 23, "pos"]	319
[28, 3, "neg"]	153
[28, 3, "neu"]	608
[28, 3, "pos"]	262
[28, 4, "neg"]	163
[28, 4, "neu"]	635
[28, 4, "pos"]	229
[28, 5, "neg"]	156
[28, 5, "neu"]	615
[28, 5, "pos"]	216
[28, 6, "neg"]	153
[28, 6, "neu"]	664
[28, 6, "pos"]	267
[28, 7, "neg"]	175
[28, 7, "neu"]	724
[28, 7, "pos"]	259
[28, 8, "neg"]	201
[28, 8, "neu"]	773
[28, 8, "pos"]	390
[28, 9, "neg"]	228
[28, 9, "neu"]	979
[28, 9, "pos"]	496
[29, 0, "neg"]	173
[29, 0, "neu"]	579
[29, 0, "pos"]	463
[29, 1, "neg"]	113
[29, 1, "neu"]	512
[29, 1, "pos"]	294
[29, 10, "neg"]	309
[29, 10, "neu"]	1190
[29, 10, "pos"]	628
[29, 11, "neg"]	508
[29, 11, "neu"]	1078
[29, 11, "pos"]	657
[29, 12, "neg"]	351
[29, 12, "neu"]	1128
[29, 12, "pos"]	671
[29, 13, "neg"]	257
[29, 13, "neu"]	877
[29, 13, "pos"]	581
[29, 14, "neg"]	244
[29, 14, "neu"]	1000
[29, 14, "pos"]	482
[29, 15, "neg"]	265
[29, 15, "neu"]	860
[29, 15, "pos"]	514
[29, 16, "neg"]	326
[29, 16, "neu"]	840
[29, 16, "pos"]	539
[29, 17, "neg"]	210
[29, 17, "neu"]	857
[29, 17, "pos"]	492
[29, 18, "neg"]	210
[29, 18, "neu"]	762
[29, 18, "pos"]	474
[29, 19, "neg"]	204
[29, 19, "neu"]	700
[29, 19, "pos"]	427
[29, 2, "neg"]	137
[29, 2, "neu"]	512
[29, 2, "pos"]	312
[29, 20, "neg"]	156
[29, 20, "neu"]	585
[29, 20, "pos"]	611
[29, 21, "neg"]	161
[29, 21, "neu"]	643
[29, 21, "pos"]	363
[29, 22, "neg"]	128
[29, 22, "neu"]	605
[29, 22, "pos"]	362
[29, 23, "neg"]	205
[29, 23, "neu"]	488
[29, 23, "pos"]	296
[29, 3, "neg"]	133
[29, 3, "neu"]	552
[29, 3, "pos"]	228
[29, 4, "neg"]	120
[29, 4, "neu"]	766
[29, 4, "pos"]	352
[29, 5, "neg"]	149
[29, 5, "neu"]	881
[29, 5, "pos"]	321
[29, 6, "neg"]	159
[29, 6, "neu"]	883
[29, 6, "pos"]	410
[29, 7, "neg"]	168
[29, 7, "neu"]	850
[29, 7, "pos"]	339
[29, 8, "neg"]	223
[29, 8, "neu"]	843
[29, 8, "pos"]	298
[29, 9, "neg"]	223
[29, 9, "neu"]	1036
[29, 9, "pos"]	446
[30, 0, "neg"]	107
[30, 0, "neu"]	474
[30, 0, "pos"]	236
[30, 1, "neg"]	127
[30, 1, "neu"]	502
[30, 1, "pos"]	266
[30, 10, "neg"]	216
[30, 10, "neu"]	881
[30, 10, "pos"]	446
[30, 11, "neg"]	217
[30, 11, "neu"]	768
[30, 11, "pos"]	453
[30, 12, "neg"]	243
[30, 12, "neu"]	886
[30, 12, "pos"]	593
[30, 13, "neg"]	232
[30, 13, "neu"]	755
[30, 13, "pos"]	508
[30, 14, "neg"]	240
[30, 14, "neu"]	668
[30, 14, "pos"]	456
[30, 15, "neg"]	229
[30, 15, "neu"]	687
[30, 15, "pos"]	448
[30, 16, "neg"]	194
[30, 16, "neu"]	738
[30, 16, "pos"]	419
[30, 17, "neg"]	186
[30, 17, "neu"]	598
[30, 17, "pos"]	379
[30, 18, "neg"]	183
[30, 18, "neu"]	645
[30, 18, "pos"]	401
[30, 19, "neg"]	185
[30, 19, "neu"]	594
[30, 19, "pos"]	342
[30, 2, "neg"]	118
[30, 2, "neu"]	457
[30, 2, "pos"]	259
[30, 20, "neg"]	141
[30, 20, "neu"]	565
[30, 20, "pos"]	321
[30, 21, "neg"]	135
[30, 21, "neu"]	563
[30, 21, "pos"]	269
[30, 22, "neg"]	125
[30, 22, "neu"]	542
[30, 22, "pos"]	271
[30, 23, "neg"]	130
[30, 23, "neu"]	519
[30, 23, "pos"]	282
[30, 3, "neg"]	133
[30, 3, "neu"]	568
[30, 3, "pos"]	280
[30, 4, "neg"]	133
[30, 4, "neu"]	704
[30, 4, "pos"]	280
[30, 5, "neg"]	151
[30, 5, "neu"]	648
[30, 5, "pos"]	288
[30, 6, "neg"]	184
[30, 6, "neu"]	645
[30, 6, "pos"]	264
[30, 7, "neg"]	265
[30, 7, "neu"]	662
[30, 7, "pos"]	283
[30, 8, "neg"]	164
[30, 8, "neu"]	635
[30, 8, "pos"]	284
[30, 9, "neg"]	188
[30, 9, "neu"]	774
[30, 9, "pos"]	403
[31, 0, "neg"]	104
[31, 0, "neu"]	504
[31, 0, "pos"]	293
[31, 1, "neg"]	121
[31, 1, "neu"]	448
[31, 1, "pos"]	312
[31, 10, "neg"]	194
[31, 10, "neu"]	818
[31, 10, "pos"]	436
[31, 11, "neg"]	245
[31, 11, "neu"]	787
[31, 11, "pos"]	450
[31, 12, "neg"]	199
[31, 12, "neu"]	867
[31, 12, "pos"]	489
[31, 13, "neg"]	253
[31, 13, "neu"]	778
[31, 13, "pos"]	484
[31, 14, "neg"]	254
[31, 14, "neu"]	895
[31, 14, "pos"]	487
[31, 15, "neg"]	283
[31, 15, "neu"]	804
[31, 15, "pos"]	495
[31, 16, "neg"]	273
[31, 16, "neu"]	755
[31, 16, "pos"]	514
[31, 17, "neg"]	271
[31, 17, "neu"]	766
[31, 17, "pos"]	436
[31, 18, "neg"]	188
[31, 18, "neu"]	814
[31, 18, "pos"]	411
[31, 19, "neg"]	192
[31, 19, "neu"]	679
[31, 19, "pos"]	363
[31, 2, "neg"]	109
[31, 2, "neu"]	436
[31, 2, "pos"]	213
[31, 20, "neg"]	178
[31, 20, "neu"]	571
[31, 20, "pos"]	308
[31, 21, "neg"]	143
[31, 21, "neu"]	601
[31, 21, "pos"]	297
[31, 22, "neg"]	143
[31, 22, "neu"]	578
[31, 22, "pos"]	299
[31, 23, "neg"]	125
[31, 23, "neu"]	521
[31, 23, "pos"]	544
[31, 3, "neg"]	132
[31, 3, "neu"]	585
[31, 3, "pos"]	210
[31, 4, "neg"]	156
[31, 4, "neu"]	652
[31, 4, "pos"]	258
[31, 5, "neg"]	138
[31, 5, "neu"]	608
[31, 5, "pos"]	262
[31, 6, "neg"]	126
[31, 6, "neu"]	604
[31, 6, "pos"]	279
[31, 7, "neg"]	168
[31, 7, "neu"]	656
[31, 7, "pos"]	299
[31, 8, "neg"]	160
[31, 8, "neu"]	646
[31, 8, "pos"]	325
[31, 9, "neg"]	162
[31, 9, "neu"]	807
[31, 9, "pos"]	316
[32, 0, "neg"]	111
[32, 0, "neu"]	420
[32, 0, "pos"]	186
[32, 1, "neg"]	165
[32, 1, "neu"]	533
[32, 1, "pos"]	253
[32, 10, "neg"]	270
[32, 10, "neu"]	719
[32, 10, "pos"]	561
[32, 11, "neg"]	234
[32, 11, "neu"]	753
[32, 11, "pos"]	567
[32, 12, "neg"]	283
[32, 12, "neu"]	927
[32, 12, "pos"]	580
[32, 13, "neg"]	383
[32, 13, "neu"]	747
[32, 13, "pos"]	464
[32, 14, "neg"]	309
[32, 14, "neu"]	757
[32, 14, "pos"]	543
[32, 15, "neg"]	280
[32, 15, "neu"]	811
[32, 15, "pos"]	421
[32, 16, "neg"]	284
[32, 16, "neu"]	743
[32, 16, "pos"]	364
[32, 17, "neg"]	282
[32, 17, "neu"]	704
[32, 17, "pos"]	388
[32, 18, "neg"]	194
[32, 18, "neu"]	655
[32, 18, "pos"]	394
[32, 19, "neg"]	177
[32, 19, "neu"]	683
[32, 19, "pos"]	343
[32, 2, "neg"]	117
[32, 2, "neu"]	507
[32, 2, "pos"]	271
[32, 20, "neg"]	180
[32, 20, "neu"]	615
[32, 20, "pos"]	328
[32, 21, "neg"]	188
[32, 21, "neu"]	638
[32, 21, "pos"]	293
[32, 22, "neg"]	173
[32, 22, "neu"]	621
[32, 22, "pos"]	303
[32, 23, "neg"]	150
[32, 23, "neu"]	492
[32, 23, "pos"]	303
[32, 3, "neg"]	123
[32, 3, "neu"]	471
[32, 3, "pos"]	255
[32, 4, "neg"]	201
[32, 4, "neu"]	584
[32, 4, "pos"]	223
[32, 5, "neg"]	284
[32, 5, "neu"]	556
[32, 5, "pos"]	223
[32, 6, "neg"]	189
[32, 6, "neu"]	576
[32, 6, "pos"]	242
[32, 7, "neg"]	315
[32, 7, "neu"]	610
[32, 7, "pos"]	283
[32, 8, "neg"]	167
[32, 8, "neu"]	579
[32, 8, "pos"]	265
[32, 9, "neg"]	219
[32, 9, "neu"]	639
[32, 9, "pos"]	341
[33, 0, "neg"]	131
[33, 0, "neu"]	590
[33, 0, "pos"]	291
[33, 1, "neg"]	207
[33, 1, "neu"]	569
[33, 1, "pos"]	382
[33, 10, "neg"]	215
[33, 10, "neu"]	976
[33, 10, "pos"]	565
[33, 11, "neg"]	246
[33, 11, "neu"]	1004
[33, 11, "pos"]	521
[33, 12, "neg"]	286
[33, 12, "neu"]	917
[33, 12, "pos"]	502
[33, 13, "neg"]	278
[33, 13, "neu"]	928
[33, 13, "pos"]	570
[33, 14, "neg"]	275
[33, 14, "neu"]	1120
[33, 14, "pos"]	503
[33, 15, "neg"]	278
[33, 15, "neu"]	864
[33, 15, "pos"]	480
[33, 16, "neg"]	269
[33, 16, "neu"]	804
[33, 16, "pos"]	470
[33, 17, "neg"]	291
[33, 17, "neu"]	753
[33, 17, "pos"]	437
[33, 18, "neg"]	249
[33, 18, "neu"]	748
[33, 18, "pos"]	444
[33, 19, "neg"]	210
[33, 19, "neu"]	711
[33, 19, "pos"]	396
[33, 2, "neg"]	103
[33, 2, "neu"]	509
[33, 2, "pos"]	273
[33, 20, "neg"]	227
[33, 20, "neu"]	707
[33, 20, "pos"]	363
[33, 21, "neg"]	230
[33, 21, "neu"]	606
[33, 21, "pos"]	326
[33, 22, "neg"]	179
[33, 22, "neu"]	664
[33, 22, "pos"]	334
[33, 23, "neg"]	154
[33, 23, "neu"]	534
[33, 23, "pos"]	348
[33, 3, "neg"]	287
[33, 3, "neu"]	478
[33, 3, "pos"]	241
[33, 4, "neg"]	212
[33, 4, "neu"]	728
[33, 4, "pos"]	253
[33, 5, "neg"]	186
[33, 5, "neu"]	985
[33, 5, "pos"]	302
[33, 6, "neg"]	211
[33, 6, "neu"]	806
[33, 6, "pos"]	249
[33, 7, "neg"]	212
[33, 7, "neu"]	763
[33, 7, "pos"]	267
[33, 8, "neg"]	209
[33, 8, "neu"]	769
[33, 8, "pos"]	337
[33, 9, "neg"]	182
[33, 9, "neu"]	848
[33, 9, "pos"]	452
[34, 0, "neg"]	328
[34, 0, "neu"]	655
[34, 0, "pos"]	317
[34, 1, "neg"]	315
[34, 1, "neu"]	515
[34, 1, "pos"]	255
[34, 10, "neg"]	213
[34, 10, "neu"]	810
[34, 10, "pos"]	415
[34, 11, "neg"]	277
[34, 11, "neu"]	914
[34, 11, "pos"]	437
[34, 12, "neg"]	243
[34, 12, "neu"]	1065
[34, 12, "pos"]	505
[34, 13, "neg"]	290
[34, 13, "neu"]	1132
[34, 13, "pos"]	627
[34, 14, "neg"]	284
[34, 14, "neu"]	963
[34, 14, "pos"]	520
[34, 15, "neg"]	251
[34, 15, "neu"]	973
[34, 15, "pos"]	487
[34, 16, "neg"]	288
[34, 16, "neu"]	916
[34, 16, "pos"]	559
[34, 17, "neg"]	267
[34, 17, "neu"]	840
[34, 17, "pos"]	562
[34, 18, "neg"]	269
[34, 18, "neu"]	861
[34, 18, "pos"]	543
[34, 19, "neg"]	194
[34, 19, "neu"]	726
[34, 19, "pos"]	490
[34, 2, "neg"]	170
[34, 2, "neu"]	522
[34, 2, "pos"]	240
[34, 20, "neg"]	186
[34, 20, "neu"]	749
[34, 20, "pos"]	411
[34, 21, "neg"]	164
[34, 21, "neu"]	645
[34, 21, "pos"]	378
[34, 22, "neg"]	182
[34, 22, "neu"]	648
[34, 22, "pos"]	448
[34, 23, "neg"]	148
[34, 23, "neu"]	576
[34, 23, "pos"]	368
[34, 3, "neg"]	194
[34, 3, "neu"]	445
[34, 3, "pos"]	219
[34, 4, "neg"]	166
[34, 4, "neu"]	521
[34, 4, "pos"]	220
[34, 5, "neg"]	134
[34, 5, "neu"]	739
[34, 5, "pos"]	285
[34, 6, "neg"]	149
[34, 6, "neu"]	855
[34, 6, "pos"]	249
[34, 7, "neg"]	138
[34, 7, "neu"]	713
[34, 7, "pos"]	324
[34, 8, "neg"]	164
[34, 8, "neu"]	644
[34, 8, "pos"]	307
[34, 9, "neg"]	216
[34, 9, "neu"]	770
[34, 9, "pos"]	329
[35, 0, "neg"]	153
[35, 0, "neu"]	470
[35, 0, "pos"]	321
[35, 1, "neg"]	145
[35, 1, "neu"]	370
[35, 1, "pos"]	260
[35, 10, "neg"]	159
[35, 10, "neu"]	630
[35, 10, "pos"]	386
[35, 11, "neg"]	246
[35, 11, "neu"]	847
[35, 11, "pos"]	490
[35, 12, "neg"]	220
[35, 12, "neu"]	755
[35, 12, "pos"]	453
[35, 13, "neg"]	313
[35, 13, "neu"]	888
[35, 13, "pos"]	426
[35, 14, "neg"]	234
[35, 14, "neu"]	769
[35, 14, "pos"]	477
[35, 15, "neg"]	235
[35, 15, "neu"]	675
[35, 15, "pos"]	447
[35, 16, "neg"]	228
[35, 16, "neu"]	737
[35, 16, "pos"]	418
[35, 17, "neg"]	182
[35, 17, "neu"]	653
[35, 17, "pos"]	432
[35, 18, "neg"]	201
[35, 18, "neu"]	653
[35, 18, "pos"]	453
[35, 19, "neg"]	175
[35, 19, "neu"]	556
[35, 19, "pos"]	390
[35, 2, "neg"]	121
[35, 2, "neu"]	433
[35, 2, "pos"]	245
[35, 20, "neg"]	177
[35, 20, "neu"]	574
[35, 20, "pos"]	383
[35, 21, "neg"]	179
[35, 21, "neu"]	515
[35, 21, "pos"]	308
[35, 22, "neg"]	168
[35, 22, "neu"]	535
[35, 22, "pos"]	399
[35, 23, "neg"]	130
[35, 23, "neu"]	450
[35, 23, "pos"]	292
[35, 3, "neg"]	142
[35, 3, "neu"]	505
[35, 3, "pos"]	273
[35, 4, "neg"]	138
[35, 4, "neu"]	524
[35, 4, "pos"]	228
[35, 5, "neg"]	208
[35, 5, "neu"]	477
[35, 5, "pos"]	253
[35, 6, "neg"]	169
[35, 6, "neu"]	549
[35, 6, "pos"]	277
[35, 7, "neg"]	139
[35, 7, "neu"]	540
[35, 7, "pos"]	237
[35, 8, "neg"]	438
[35, 8, "neu"]	558
[35, 8, "pos"]	323
[35, 9, "neg"]	241
[35, 9, "neu"]	605
[35, 9, "pos"]	291
[36, 0, "neg"]	115
[36, 0, "neu"]	386
[36, 0, "pos"]	232
[36, 1, "neg"]	135
[36, 1, "neu"]	443
[36, 1, "pos"]	282
[36, 10, "neg"]	149
[36, 10, "neu"]	604
[36, 10, "pos"]	319
[36, 11, "neg"]	155
[36, 11, "neu"]	557
[36, 11, "pos"]	321
[36, 12, "neg"]	147
[36, 12, "neu"]	595
[36, 12, "pos"]	353
[36, 13, "neg"]	205
[36, 13, "neu"]	726
[36, 13, "pos"]	428
[36, 14, "neg"]	192
[36, 14, "neu"]	666
[36, 14, "pos"]	369
[36, 15, "neg"]	306
[36, 15, "neu"]	591
[36, 15, "pos"]	358
[36, 16, "neg"]	196
[36, 16, "neu"]	712
[36, 16, "pos"]	438
[36, 17, "neg"]	159
[36, 17, "neu"]	549
[36, 17, "pos"]	340
[36, 18, "neg"]	163
[36, 18, "neu"]	525
[36, 18, "pos"]	391
[36, 19, "neg"]	183
[36, 19, "neu"]	511
[36, 19, "pos"]	404
[36, 2, "neg"]	104
[36, 2, "neu"]	307
[36, 2, "pos"]	176
[36, 20, "neg"]	180
[36, 20, "neu"]	588
[36, 20, "pos"]	333
[36, 21, "neg"]	164
[36, 21, "neu"]	463
[36, 21, "pos"]	296
[36, 22, "neg"]	253
[36, 22, "neu"]	1077
[36, 22, "pos"]	543
[36, 23, "neg"]	333
[36, 23, "neu"]	1360
[36, 23, "pos"]	548
[36, 3, "neg"]	102
[36, 3, "neu"]	371
[36, 3, "pos"]	195
[36, 4, "neg"]	90
[36, 4, "neu"]	427
[36, 4, "pos"]	213
[36, 5, "neg"]	102
[36, 5, "neu"]	371
[36, 5, "pos"]	238
[36, 6, "neg"]	93
[36, 6, "neu"]	449
[36, 6, "pos"]	207
[36, 7, "neg"]	149
[36, 7, "neu"]	531
[36, 7, "pos"]	254
[36, 8, "neg"]	117
[36, 8, "neu"]	390
[36, 8, "pos"]	226
[36, 9, "neg"]	130
[36, 9, "neu"]	463
[36, 9, "pos"]	259
[37, 0, "neg"]	462
[37, 0, "neu"]	1358
[37, 0, "pos"]	1119
[37, 1, "neg"]	469
[37, 1, "neu"]	1384
[37, 1, "pos"]	1051
[37, 10, "neg"]	215
[37, 10, "neu"]	1144
[37, 10, "pos"]	448
[37, 11, "neg"]	141
[37, 11, "neu"]	495
[37, 11, "pos"]	344
[37, 12, "neg"]	173
[37, 12, "neu"]	575
[37, 12, "pos"]	408
[37, 13, "neg"]	386
[37, 13, "neu"]	1216
[37, 13, "pos"]	869
[37, 14, "neg"]	479
[37, 14, "neu"]	1989
[37, 14, "pos"]	1142
[37, 15, "neg"]	475
[37, 15, "neu"]	2113
[37, 15, "pos"]	1243
[37, 16, "neg"]	183
[37, 16, "neu"]	513
[37, 16, "pos"]	429
[37, 17, "neg"]	172
[37, 17, "neu"]	488
[37, 17, "pos"]	423
[37, 18, "neg"]	168
[37, 18, "neu"]	530
[37, 18, "pos"]	438
[37, 19, "neg"]	146
[37, 19, "neu"]	458
[37, 19, "pos"]	394
[37, 2, "neg"]	528
[37, 2, "neu"]	2344
[37, 2, "pos"]	1275
[37, 20, "neg"]	143
[37, 20, "neu"]	424
[37, 20, "pos"]	357
[37, 21, "neg"]	181
[37, 21, "neu"]	478
[37, 21, "pos"]	386
[37, 22, "neg"]	144
[37, 22, "neu"]	409
[37, 22, "pos"]	353
[37, 23, "neg"]	116
[37, 23, "neu"]	346
[37, 23, "pos"]	264
[37, 3, "neg"]	425
[37, 3, "neu"]	1637
[37, 3, "pos"]	930
[37, 4, "neg"]	405
[37, 4, "neu"]	1467
[37, 4, "pos"]	821
[37, 5, "neg"]	343
[37, 5, "neu"]	1206
[37, 5, "pos"]	712
[37, 6, "neg"]	311
[37, 6, "neu"]	1181
[37, 6, "pos"]	652
[37, 7, "neg"]	231
[37, 7, "neu"]	1032
[37, 7, "pos"]	521
[37, 8, "neg"]	210
[37, 8, "neu"]	1221
[37, 8, "pos"]	410
[37, 9, "neg"]	224
[37, 9, "neu"]	1215
[37, 9, "pos"]	474
[38, 0, "neg"]	146
[38, 0, "neu"]	418
[38, 0, "pos"]	301
[38, 1, "neg"]	108
[38, 1, "neu"]	304
[38, 1, "pos"]	235
[38, 10, "neg"]	148
[38, 10, "neu"]	464
[38, 10, "pos"]	327
[38, 11, "neg"]	159
[38, 11, "neu"]	627
[38, 11, "pos"]	334
[38, 12, "neg"]	168
[38, 12, "neu"]	548
[38, 12, "pos"]	371
[38, 13, "neg"]	171
[38, 13, "neu"]	550
[38, 13, "pos"]	406
[38, 14, "neg"]	193
[38, 14, "neu"]	614
[38, 14, "pos"]	371
[38, 15, "neg"]	179
[38, 15, "neu"]	525
[38, 15, "pos"]	392
[38, 16, "neg"]	162
[38, 16, "neu"]	578
[38, 16, "pos"]	431
[38, 17, "neg"]	151
[38, 17, "neu"]	520
[38, 17, "pos"]	395
[38, 18, "neg"]	174
[38, 18, "neu"]	505
[38, 18, "pos"]	388
[38, 19, "neg"]	134
[38, 19, "neu"]	422
[38, 19, "pos"]	315
[38, 2, "neg"]	112
[38, 2, "neu"]	299
[38, 2, "pos"]	189
[38, 20, "neg"]	152
[38, 20, "neu"]	420
[38, 20, "pos"]	333
[38, 21, "neg"]	131
[38, 21, "neu"]	420
[38, 21, "pos"]	353
[38, 22, "neg"]	141
[38, 22, "neu"]	421
[38, 22, "pos"]	300
[38, 23, "neg"]	125
[38, 23, "neu"]	451
[38, 23, "pos"]	305
[38, 3, "neg"]	130
[38, 3, "neu"]	356
[38, 3, "pos"]	236
[38, 4, "neg"]	112
[38, 4, "neu"]	356
[38, 4, "pos"]	168
[38, 5, "neg"]	112
[38, 5, "neu"]	369
[38, 5, "pos"]	190
[38, 6, "neg"]	112
[38, 6, "neu"]	384
[38, 6, "pos"]	217
[38, 7, "neg"]	118
[38, 7, "neu"]	340
[38, 7, "pos"]	198
[38, 8, "neg"]	107
[38, 8, "neu"]	423
[38, 8, "pos"]	212
[38, 9, "neg"]	129
[38, 9, "neu"]	482
[38, 9, "pos"]	236
[39, 0, "neg"]	506
[39, 0, "neu"]	2687
[39, 0, "pos"]	935
[39, 1, "neg"]	419
[39, 1, "neu"]	2694
[39, 1, "pos"]	826
[39, 10, "neg"]	509
[39, 10, "neu"]	2298
[39, 10, "pos"]	809
[39, 11, "neg"]	101
[39, 11, "neu"]	353
[39, 11, "pos"]	218
[39, 12, "neg"]	99
[39, 12, "neu"]	319
[39, 12, "pos"]	200
[39, 13, "neg"]	173
[39, 13, "neu"]	819
[39, 13, "pos"]	361
[39, 14, "neg"]	443
[39, 14, "neu"]	2812
[39, 14, "pos"]	810
[39, 15, "neg"]	844
[39, 15, "neu"]	5347
[39, 15, "pos"]	1511
[39, 16, "neg"]	826
[39, 16, "neu"]	4765
[39, 16, "pos"]	1546
[39, 17, "neg"]	524
[39, 17, "neu"]	2809
[39, 17, "pos"]	1008
[39, 18, "neg"]	588
[39, 18, "neu"]	3040
[39, 18, "pos"]	1024
[39, 19, "neg"]	463
[39, 19, "neu"]	2042
[39, 19, "pos"]	838
[39, 2, "neg"]	430
[39, 2, "neu"]	2729
[39, 2, "pos"]	719
[39, 20, "neg"]	490
[39, 20, "neu"]	2041
[39, 20, "pos"]	750
[39, 21, "neg"]	266
[39, 21, "neu"]	1417
[39, 21, "pos"]	546
[39, 22, "neg"]	432
[39, 22, "neu"]	1971
[39, 22, "pos"]	795
[39, 23, "neg"]	809
[39, 23, "neu"]	3296
[39, 23, "pos"]	1165
[39, 3, "neg"]	386
[39, 3, "neu"]	3031
[39, 3, "pos"]	791
[39, 4, "neg"]	396
[39, 4, "neu"]	2652
[39, 4, "pos"]	751
[39, 5, "neg"]	420
[39, 5, "neu"]	2655
[39, 5, "pos"]	616
[39, 6, "neg"]	245
[39, 6, "neu"]	1815
[39, 6, "pos"]	511
[39, 7, "neg"]	296
[39, 7, "neu"]	1548
[39, 7, "pos"]	466
[39, 8, "neg"]	468
[39, 8, "neu"]	2303
[39, 8, "pos"]	696
[39, 9, "neg"]	463
[39, 9, "neu"]	2401
[39, 9, "pos"]	720
[40, 0, "neg"]	289
[40, 0, "neu"]	1972
[40, 0, "pos"]	815
[40, 1, "neg"]	377
[40, 1, "neu"]	2658
[40, 1, "pos"]	1044
[40, 10, "neg"]	230
[40, 10, "neu"]	2103
[40, 10, "pos"]	592
[40, 11, "neg"]	82
[40, 11, "neu"]	1066
[40, 11, "pos"]	306
[40, 12, "neg"]	74
[40, 12, "neu"]	1075
[40, 12, "pos"]	277
[40, 13, "neg"]	107
[40, 13, "neu"]	1241
[40, 13, "pos"]	329
[40, 14, "neg"]	275
[40, 14, "neu"]	2443
[40, 14, "pos"]	636
[40, 15, "neg"]	306
[40, 15, "neu"]	2996
[40, 15, "pos"]	760
[40, 16, "neg"]	607
[40, 16, "neu"]	5287
[40, 16, "pos"]	1426
[40, 17, "neg"]	793
[40, 17, "neu"]	4807
[40, 17, "pos"]	1663
[40, 18, "neg"]	515
[40, 18, "neu"]	3050
[40, 18, "pos"]	1094
[40, 19, "neg"]	334
[40, 19, "neu"]	1977
[40, 19, "pos"]	795
[40, 2, "neg"]	610
[40, 2, "neu"]	3269
[40, 2, "pos"]	1242
[40, 20, "neg"]	317
[40, 20, "neu"]	2229
[40, 20, "pos"]	900
[40, 21, "neg"]	336
[40, 21, "neu"]	2176
[40, 21, "pos"]	931
[40, 22, "neg"]	526
[40, 22, "neu"]	2111
[40, 22, "pos"]	1157
[40, 23, "neg"]	485
[40, 23, "neu"]	2066
[40, 23, "pos"]	905
[40, 3, "neg"]	611
[40, 3, "neu"]	4413
[40, 3, "pos"]	1273
[40, 4, "neg"]	395
[40, 4, "neu"]	3130
[40, 4, "pos"]	914
[40, 5, "neg"]	366
[40, 5, "neu"]	2389
[40, 5, "pos"]	875
[40, 6, "neg"]	258
[40, 6, "neu"]	2347
[40, 6, "pos"]	632
[40, 7, "neg"]	388
[40, 7, "neu"]	2577
[40, 7, "pos"]	795
[40, 8, "neg"]	281
[40, 8, "neu"]	2794
[40, 8, "pos"]	786
[40, 9, "neg"]	257
[40, 9, "neu"]	2836
[40, 9, "pos"]	659
[41, 0, "neg"]	643
[41, 0, "neu"]	3360
[41, 0, "pos"]	1328
[41, 1, "neg"]	480
[41, 1, "neu"]	2982
[41, 1, "pos"]	1319
[41, 10, "neg"]	341
[41, 10, "neu"]	3194
[41, 10, "pos"]	780
[41, 11, "neg"]	320
[41, 11, "neu"]	2263
[41, 11, "pos"]	711
[41, 12, "neg"]	346
[41, 12, "neu"]	2564
[41, 12, "pos"]	700
[41, 13, "neg"]	550
[41, 13, "neu"]	4002
[41, 13, "pos"]	1049
[41, 14, "neg"]	750
[41, 14, "neu"]	6186
[41, 14, "pos"]	1809
[41, 15, "neg"]	736
[41, 15, "neu"]	6381
[41, 15, "pos"]	1901
[41, 16, "neg"]	1069
[41, 16, "neu"]	8058
[41, 16, "pos"]	2373
[41, 17, "neg"]	776
[41, 17, "neu"]	4713
[41, 17, "pos"]	1716
[41, 18, "neg"]	609
[41, 18, "neu"]	3313
[41, 18, "pos"]	1271
[41, 19, "neg"]	595
[41, 19, "neu"]	3256
[41, 19, "pos"]	1608
[41, 2, "neg"]	881
[41, 2, "neu"]	3958
[41, 2, "pos"]	1651
[41, 20, "neg"]	596
[41, 20, "neu"]	3257
[41, 20, "pos"]	1462
[41, 21, "neg"]	509
[41, 21, "neu"]	2912
[41, 21, "pos"]	1122
[41, 22, "neg"]	881
[41, 22, "neu"]	5324
[41, 22, "pos"]	2229
[41, 23, "neg"]	731
[41, 23, "neu"]	4405
[41, 23, "pos"]	1845
[41, 3, "neg"]	650
[41, 3, "neu"]	4233
[41, 3, "pos"]	1643
[41, 4, "neg"]	706
[41, 4, "neu"]	3930
[41, 4, "pos"]	1539
[41, 5, "neg"]	582
[41, 5, "neu"]	3754
[41, 5, "pos"]	1411
[41, 6, "neg"]	516
[41, 6, "neu"]	3409
[41, 6, "pos"]	1374
[41, 7, "neg"]	454
[41, 7, "neu"]	3546
[41, 7, "pos"]	1062
[41, 8, "neg"]	368
[41, 8, "neu"]	3599
[41, 8, "pos"]	1185
[41, 9, "neg"]	410
[41, 9, "neu"]	4535
[41, 9, "pos"]	1248
[42, 0, "neg"]	432
[42, 0, "neu"]	2878
[42, 0, "pos"]	870
[42, 1, "neg"]	443
[42, 1, "neu"]	2744
[42, 1, "pos"]	837
[42, 13, "neg"]	96
[42, 13, "neu"]	912
[42, 13, "pos"]	234
[42, 14, "neg"]	181
[42, 14, "neu"]	1247
[42, 14, "pos"]	396
[42, 15, "neg"]	32
[42, 15, "neu"]	212
[42, 15, "pos"]	46
[42, 16, "neg"]	137
[42, 16, "neu"]	1111
[42, 16, "pos"]	336
[42, 17, "neg"]	275
[42, 17, "neu"]	1301
[42, 17, "pos"]	673
[42, 18, "neg"]	112
[42, 18, "neu"]	638
[42, 18, "pos"]	238
[42, 2, "neg"]	504
[42, 2, "neu"]	3008
[42, 2, "pos"]	1015
[42, 3, "neg"]	206
[42, 3, "neu"]	1256
[42, 3, "pos"]	387


Time taken to completion of the metric: 1356.91702487 in processor time + X

**Analysis**

From this we can see which hours in which week had more positive or negative records by platform.


**Graphic**

In [1]:
import matplotlib.pyplot as plt
archivo = open("sentiment_analysis_by_platform_summary.txt")
archivo.readline()
pos = []
neg = []
neu = []
chars_to_remove = [',', '[', ']', '"']
for i in archivo:
	line = i.translate(None, ''.join(chars_to_remove))
	line = line.split()
	if line[2] == "pos":
		pos.append([int(line[0]), int(line[1]), int(line[3])])
	elif line[2] == "neg":
		neg.append([int(line[0]), int(line[1]), int(line[3])])
	else:
		neu.append([int(line[0]), int(line[1]), int(line[3])])

x_neg = []
x_neg_name = []
y_neg = []
count = 0
for i in sorted(neg):
	x_neg.append(count)
	x_neg_name.append(str(i[0])+ " " + str(i[1]))
	y_neg.append(i[2])
	count += 1

plt.xticks(x_neg, x_neg_name)
neg_plt = plt.plot(x_neg, y_neg, 'r',label='negatives')

x_neu = []
x_neu_name = []
y_neu = []
count = 0
for i in sorted(neu):
	x_neu.append(count)
	x_neu_name.append(str(i[0])+ " " + str(i[1]))
	y_neu.append(i[2])
	count += 1

plt.xticks(x_neu, x_neu_name)
neu_plt = plt.plot(x_neu, y_neu, 'b',label='neutral')


x_pos = []
x_pos_name = []
y_pos = []
count = 0
for i in sorted(pos):
	x_pos.append(count)
	x_pos_name.append(str(i[0])+ " " + str(i[1]))
	y_pos.append(i[2])
	count += 1

plt.xticks(x_pos, x_pos_name)
pos_plt = plt.plot(x_pos, y_pos, 'g', label='positive')
plt.legend(loc=2)
plt.show()

IndexError: list index out of range

5.3) We now calculate how many records are from the different languages for each platform with the next script.

In [ ]:
import time
import sys
import language_detect as ld

# The summary is for each hour per week. (i.e. 20 spanish tweets on week 1, 31 english tweets on week 2, ...)

# Important Note: This is just an estimation of the language based on the stopwords, so if no stopwords are found the language
#                   that will produce is garbage, but is not the fault of the model. Also if there are some stopwords of both
#                   languages it will say is the one with most stopwords.

if __name__ == '__main__':
    time_start = time.clock()
    # Clean File.
    open("languages_amount_by_platform_summary.txt", 'w').close()
    files = ["project_tweets02.data", "project_tweets03.data", "project_tweets04.data"]
    for _file in files:
        _output_filename = ld.set_language_for_each_tweet(_file)
        # parameters for mrjob.
        # To run your job in multiple subprocesses with a few Hadoop features simulated, use -r local.
        option1 = "" #""-r"
        option2 = "" #""local"
        sys.argv = ['count_language.py', option1, option2, _output_filename]
        # Write to file in append mode.
        _fo = open("languages_amount_by_platform_summary.txt", 'a')
        sys.stdout = _fo
        print _file
        execfile('count_language.py')
        print "\n"

    time_end = time.clock()
    print "Time taken to completion of the metric: {0} in processor time".format(time_end - time_start)


In [ ]:
import sys
import json
reload(sys)
sys.setdefaultencoding('utf8')
try:
    from nltk import wordpunct_tokenize
    from nltk.corpus import stopwords
except ImportError:
    print '[!] You need to install nltk (http://nltk.org/index.html)'



#----------------------------------------------------------------------
def _calculate_languages_ratios(text):
    """
    Calculate probability of given text to be written in several languages and
    return a dictionary that looks like {'french': 2, 'spanish': 4, 'english': 0}
    
    @param text: Text whose language want to be detected
    @type text: str
    
    @return: Dictionary with languages and unique stopwords seen in analyzed text
    @rtype: dict
    """

    languages_ratios = {}

    '''
    nltk.wordpunct_tokenize() splits all punctuations into separate tokens
    
    >>> wordpunct_tokenize("That's thirty minutes away. I'll be there in ten.")
    ['That', "'", 's', 'thirty', 'minutes', 'away', '.', 'I', "'", 'll', 'be', 'there', 'in', 'ten', '.']
    '''

    tokens = wordpunct_tokenize(text)
    words = [word.lower() for word in tokens]

    # Compute per language included in nltk number of unique stopwords appearing in analyzed text
    for language in stopwords.fileids():
        stopwords_set = set(stopwords.words(language))
        words_set = set(words)
        common_elements = words_set.intersection(stopwords_set)

        languages_ratios[language] = len(common_elements) # language "score"

    return languages_ratios


#----------------------------------------------------------------------
def detect_language(text):
    """
    Calculate probability of given text to be written in several languages and
    return the highest scored.
    
    It uses a stopwords based approach, counting how many unique stopwords
    are seen in analyzed text.
    
    @param text: Text whose language want to be detected
    @type text: str
    
    @return: Most scored language guessed
    @rtype: str
    """

    ratios = _calculate_languages_ratios(text)

    most_rated_language = max(ratios, key=ratios.get)

    return most_rated_language


def set_language_for_each_tweet(_file):
    _data = open(_file, 'r')
    open(_file[:-5] + "_language.data", 'w').close()
    _output = open(_file[:-5] + "_language.data", 'a')
    for line in _data.readlines():
        record = json.loads(line)
        language = detect_language(record['text'].encode('utf-8'))
        try:
            record.update({'language': language})
            _output.write(json.dumps(record, sort_keys=True) + "\n")
        except:
            # Should not happen.
            print "WTF!"
    _data.close()
    _output.close()
    return _output.name



In [ ]:
from mrjob.job import MRJob
from mrjob.step import MRStep
from mrjob.protocol import JSONValueProtocol
from datetime import datetime
import itertools


class MRWordFrequencyCount(MRJob):
    INPUT_PROTOCOL = JSONValueProtocol

    def mapper(self, _, record):
        yield [record['language'], 1]

    def reducer(self, key, values):
        yield [key, sum(values)]

    def steps(self):
        return [MRStep(mapper=self.mapper, reducer=self.max_reducer)]

if __name__ == '__main__':
    MRWordFrequencyCount.run()


**Results**



**Analysis**

From this we can see which hours in which week had more records made in the different languages by platform.
